---
# CUDA Programming in C
---

This notebook contains an introduction to CUDA programming in C. For detailed coverage, NVidia's documentation is a good source:

- [CUDA C Programming Guide](https://docs.nvidia.com/cuda/cuda-c-programming-guide/contents.html)
- [Nsight Compute CLI](https://docs.nvidia.com/nsight-compute/NsightComputeCli/index.html)
- [Nsight Developer Tools](https://docs.nvidia.com/nsight-developer-tools/index.html)

**Note! If you don't have an Nvidia GPU on your system**

- You can run this notebook in Google CoLab.
  - Skip ahead to [1.1 Running the Notebook Locally or on Google CoLab](#11-running-the-notebook-locally-or-on-google-coLab)

**Note! If you are on Windows**

- Make sure you have installed Visual Studio or the Build Tools for Visual Studio.
- Make sure you have started VSCode (`code .`) from within a `Visual Studio Developer Command Prompt` to set necessary environment variables.
  - This is required when using the MicroSoft Visual C/C++ (MSVC) compiler `cl.exe` in VSCode on Windows.
  - The NVidia Cuda Compiler (NVCC) compiles `device` code, but uses your system's default C/C++ compiler to compile `host` code (GCC on Linux, MSVC on Windows).
  - If you have multiple C/C++ compilers installed on your system, you can tell NVCC which C/C++ compiler to use with the flag `--compiler-bindir <PATH_TO_DIR>`, where `<PATH_TO_DIR>` is the path to your C/C++ compiler's root directory.

This notebook covers:

- [1. Prerequisites](#1-prerequisites) 
  - [1.1 Running the Notebook Locally or on Google CoLab](#11-running-the-notebook-locally-or-on-google-coLab)
  - [1.2 C Compiler (`gcc`, `clang`, `cl`) and CUDA Comiler (`nvcc`)](#12-c-compiler-gcc-clang-cl-and-cuda-compiler-nvcc)
  - [1.3 Configuring `tasks.json`, `launch.json` and `c_cpp_properties.json`](#13-configuring-tasksjson-launchjson-and-c_cpp_propertiesjson)
  - [1.4 Create the File `tasks.json`](#14-create-the-file-tasksjson)
  - [1.5 Create the File `launch.json`](#15-create-the-file-launchjson)
  - [1.6 Create the File `c_cpp_properties.json`](#16-create-the-file-c_cpp_propertiesjson)
  - [1.7 VSCode Extensions](#17-vscode-extensions)
  - [1.8 Using Notebook Extension `nvcc4jupyter` and Cell Magic `%%cuda`](#18-using-notebook-extension-nvcc4jupyter-and-cell-magic-cuda)
  - [1.9 Using Built-in Cell Magic `%%writefile`](#19-using-built-in-cell-magic-writefile)
  - [1.10 Compiling and Executing a CUDA Program from a Notebook Code Cell](#110-compiling-and-executing-a-cuda-program-from-a-notebook-code-cell)
  - [1.11 Compiling and Debugging a Single-file CUDA Program](#111-compiling-and-debugging-a-single-file-cuda-program)
  - [1.12 Compiling and Debugging a Multi-file CUDA Program](#112-compiling-and-debugging-a-multi-file-cuda-program)
- [2. CUDA Basics](#2-cuda-basics)
  - [2.1 Listing CUDA-enabled Devices and Properties](#21-listing-cuda-enabled-devices-and-properties)
  - [2.2 Hello World in Host Code (CPU)](#22-hello-world-in-host-code-cpu)
  - [2.3 Hello World in Device Code (GPU)](#23-hello-world-in-device-code-gpu)
  - [2.4 Grids, Blocks, Threads, Devices, SMs, and SPs](#24-grids-blocks-threads-devices-sms-and-sps)
  - [2.5 Unified Memory](#25-unified-memory)
  - [2.6 Error Checking](#26-error-checking)
  - [2.7 Measuring Execution Time on the Host (CPU) and on the Device (GPU)](#27-measuring-execution-time-on-the-host-cpu-and-on-the-device-gpu)
  - [2.8 Shared Memory and Thread Synchronization on the Device (GPU)](#28-shared-memory-and-thread-synchronization-on-the-device-gpu)
  - [2.9 Constant Memory on the Device (GPU)](#29-constant-memory-on-the-device-gpu)
- [3. Sample Problems](#3-sample-problems)
  - [3.1 1D Vector Addition on the Host (CPU)](#31-1d-vector-addition-on-the-host-cpu)
  - [3.2 1D Vector Addition on the Device (GPU)](#32-1d-vector-addition-on-the-device-gpu)
  - [3.3 1D Convolution on the Host (CPU)](#33-1d-convolution-on-the-host-cpu)
  - [3.4 1D Convolution on the Device (GPU)](#34-1d-convolution-on-the-device-gpu)
  - [3.5 2D Convolution on the Host (CPU)](#35-2d-convolution-on-the-host-cpu)
  - [3.6 2D Convolution on the Device (GPU)](#36-2d-convolution-on-the-device-gpu)
- [4. Cleanup](#4-cleanup)

---
# 1. Prerequisites
---

## 1.1 Running the Notebook Locally or on Google CoLab

- Run the cell below to check if you have a CUDA-enabled device and the CUDA tookit on your system.

In [1]:
!nvidia-smi
!nvcc --version

Mon Jul 14 22:41:40 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.124.06             Driver Version: 570.124.06     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX 2000 Ada Gene...    Off |   00000000:01:00.0 Off |                  N/A |
| N/A   49C    P3              9W /   35W |       1MiB /   8188MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### Inspect the output from the cell above:
- If you see an NVidia runtime `CUDA Version` (top right row) and a CUDA Toolkit `Build` version (bottom row).
  - Make sure the CUDA Toolkit `Build` version is less than or equal to the NVidia runtime `CUDA Version` (if not you can update your CUDA Toolkit or use CoLab). 
  - Then skip to [1.2 C Compiler (`gcc`, `clang`, `cl`) and CUDA Comiler (`nvcc`)](#12-c-compiler-gcc-clang-cl-and-cuda-compiler-nvcc)
- If you don't see an NVidia runtime `CUDA Version` (top right row) and a CUDA Toolkit `Build` version (bottom row), follow the instructions below.

  1. Click the icon below to open the notebook in Google CoLab.
     
     [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/paga-hb/C1PD2C_2025/blob/main/notebooks/cuda.ipynb)

  2. When the notebook opens in CoLab, choose `File -> Save a copy in Drive` from the main menu.
  3. Choose `Runtime -> Change runtime type` from the main menu, select `TP4 GPU` as the hardware accelerator, and click the `Save` button.
  4. In a notebook cell run the following code:

      ```c
      !sudo apt purge cuda && sudo apt autoremove && sudo apt autoclean
      !wget https://developer.download.nvidia.com/compute/cuda/12.4.0/local_installers/cuda_12.4.0_550.54.14_linux.run
      !chmod +x cuda_12.4.0_550.54.14_linux.run
      !sudo sh cuda_12.4.0_550.54.14_linux.run --silent --toolkit
      !pip install nvcc4jupyter
      %load_ext nvcc4jupyter
      !nvidia-smi
      !nvcc --version
      ```

  5. When the cell stops executing, make sure the CUDA Toolkit `Build` version is less than or equal to the NVidia runtime `CUDA Version`.
  6. Then skip to [2. CUDA Basics](#2-cuda-basics).

---

## 1.2 C Compiler (`gcc`, `clang`, `cl`) and CUDA Compiler (`nvcc`)

We are going to compose JSON configuration files for VSCode, so let's collect some information about your environment.

**Mac**

- CUDA isn't supported on Mac.
- Open the notebook in CoLab instead, and skip to Chapter 2.

**Windows**

- Your only option is to use `cl` (the C/C++ compiler, part of Microsoft Visual Studio build tools).
  - Make sure you have launched VSCode from within a `Developer Command Prompt for VS`.
    - Search in your Start Menu for `Developer Command Prompt for VS` (the version depends on your installed Visual Studio version).
    - Open it => it launches a command prompt with all environment variables (paths, includes, libs) configured to run `cl.exe` and other build tools.
    - Open VSCode from the command prompt: `code .`
  - Note that you won't be able to debug CUDA programs in VSCode (but you can in Visual Studio).
- Run the cell below to get the path to the C compiler and the CUDA compiler.
- If nothing shows up, you need to install a C/CUDA compiler (and/or make sure the C/CUDA compiler is in your `PATH` environment variable).

**Linux**

- In the cell below, choose the installed C compiler you want to use.
  - If you are using `clang` (the C compiler, part of the LLVM project).
    - Comment the row `c_compiler = "gcc"`
    - Uncomment the row `c_compiler = "clang"`
    - Note that you won't be able to debug CUDA programs in VSCode.
  - If you are using `gcc` (GNU Compiler Collection), you're all set.
- Run the cell below to get the path to the C compiler and the CUDA compiler.
- If nothing shows up, you need to install a C/CUDA compiler (and/or make sure the C/CUDA compiler is in your `PATH` environment variable).

In [2]:
c_compiler = "gcc"
#c_compiler = "clang"

import platform, os
os_name = platform.system()
if os_name == "Darwin":
    os_name = "osx"
os_name = os_name.lower()

windows_shell_name = "cmd"
#windows_shell_name = "powershell"
if os_name == 'windows':
    windows_shell_path = !where {windows_shell_name}
    windows_shell_path = windows_shell_path[0]
    windows_shell_name = os.path.basename(windows_shell_path)
    print(f"{'Windows Shell Name':<21} : {windows_shell_name}")
    print(f"{'Windows Shell Path':<21} : {windows_shell_path}")

if os_name == 'windows':
    c_compiler = "cl"
    c_compiler_path = !where {c_compiler}
else:
    c_compiler_path = !which {c_compiler}
c_compiler_path = c_compiler_path[0]
c_compiler_name = os.path.basename(c_compiler_path)

if c_compiler == 'cl':
    c_debugger_name = "cdb.exe"
    c_debugger_path = "<integrated>"
if c_compiler == "gcc":
    c_debugger_name = "gdb"
if c_compiler == "clang":
    c_debugger_name = "lldb"

if os_name == 'windows':
    if c_compiler != 'cl':
        c_debugger_path = !where {c_debugger_name}
else:
    c_debugger_path = !which {c_debugger_name}

if c_compiler != 'cl':
    c_debugger_path = c_debugger_path[0]
    c_debugger_name = os.path.basename(c_debugger_path)

cuda_compiler = "nvcc"
if os_name == 'windows':
    cuda_compiler_path = !where {cuda_compiler}
else:
    cuda_compiler_path = !which {cuda_compiler}
cuda_compiler_path = cuda_compiler_path[0]
cuda_compiler_name = os.path.basename(cuda_compiler_path)

cuda_debugger = "cuda-gdb"
if os_name == "linux" and c_compiler == "gcc":
    cuda_debugger_path = !which {cuda_debugger}
    cuda_debugger_path = cuda_debugger_path[0]
    cuda_debugger_name = os.path.basename(cuda_debugger_path)
else:
    cuda_debugger_path = "<N/A>"
    cuda_debugger_name = "<N/A>"

print(f"{'Operating System (OS)':<21} : {os_name}")
print(f"{'C Compiler Name':<21} : {c_compiler_name}")
print(f"{'C Compiler Path':<21} : {c_compiler_path}")
print(f"{'C Debugger Name':<21} : {c_debugger_name}")
print(f"{'C Debugger Path':<21} : {c_debugger_path}")
print(f"{'CUDA Compiler Name':<21} : {cuda_compiler_name}")
print(f"{'CUDA Compiler Path':<21} : {cuda_compiler_path}")
print(f"{'CUDA Debugger Name':<21} : {cuda_debugger_name}")
print(f"{'CUDA Debugger Path':<21} : {cuda_debugger_path}")

Operating System (OS) : linux
C Compiler Name       : gcc
C Compiler Path       : /usr/bin/gcc
C Debugger Name       : gdb
C Debugger Path       : /usr/bin/gdb
CUDA Compiler Name    : nvcc
CUDA Compiler Path    : /usr/local/cuda/bin/nvcc
CUDA Debugger Name    : cuda-gdb
CUDA Debugger Path    : /usr/local/cuda/bin/cuda-gdb


---
## 1.3 Configuring `tasks.json`, `launch.json`, and `c_cpp_properties.json`

- To develop CUDA programs in C with VSCode, we need to configure three VSCode workspace configuration files.
  - In the file `tasks.json` we can configure various tasks, such as build tasks for compiling and linking CUDA programs with NVCC (and our chosen C compiler).
  - In the file `launch.json` we can configure various debug options, such as debugging CUDA with CUDA-GDB (and our chosen C debugger).
  - In the file `c_cpp_properties.json` we can configure the C compiler to use for linting purposes (intellisense).
    - It isn't strictly necessary to create this configuration file to be able to run and debug CUDA programs in VSCode.
- VSCode workspace configuration files (`.json`) are stored in the subfolder `.vscode`.
- Run the cell below to create the folder `.vscode`.

**Note**

- This notebook doesn't dscribe the contents of these three files in detail. To learn more, visit: 
  - [task.json](https://code.visualstudio.com/docs/debugtest/tasks)
  - [launch.json](https://code.visualstudio.com/docs/debugtest/debugging)
  - [c_cpp_properties.json](https://code.visualstudio.com/docs/cpp/configure-intellisense)

In [3]:
import os
os.makedirs(".vscode", exist_ok=True)

---
## 1.4 Create the File `tasks.json`

- Run the cell below to create the file `tasks.json` in subfolder `.vscode`.

In [4]:
import os, json

src_path = "${workspaceFolder}/src/*.cu"
include_path = "${workspaceFolder}/include"
bin_path = "${workspaceFolder}/bin/main.exe"
if os_name == "windows":
    src_path = "${workspaceFolder}\\src\\main.cu ${workspaceFolder}\\src\\helloKernel.cu"
    include_path = "${workspaceFolder}\\include"
    bin_path = "${workspaceFolder}\\bin\\main.exe"

windows_shell_name = "windows"
makedir_command = "mkdir"
makedir_args = ["-p", "src", "include", "bin"]
if os_name == "windows":
    makedir_command = windows_shell_path
    if windows_shell_name == "powershell.exe":
        makedir_args = ["-NoProfile", "-ExecutionPolicy", "Bypass", "-Command", "New-Item -ItemType Directory -Path 'src','include','bin' -Force -ErrorAction SilentlyContinue"]
    else:
        makedir_args = ["/c", "if not exist src mkdir src & if not exist include mkdir include & if not exist bin mkdir bin"]

clean_command = "find"
clean_args = ["./bin", "-type", "f", "-name", "*.exe", "-delete"]
if os_name == "windows":
    clean_command = windows_shell_path
    if windows_shell_name == "powershell.exe":
        clean_args = ["-NoProfile", "-ExecutionPolicy", "Bypass", "-Command", "Get-ChildItem -Path .\\bin -Include *.exe, *.ilk, *.pdb, *.obj -Recurse | Remove-Item -Force"]
    else:
        clean_args = ["/c", "del /s /q /f .\\bin\\*.exe 2>nul .\\bin\\*.ilk 2>nul .\\bin\\*.pdb 2>nul .\\bin\\*.obj pdb 2>nul"]

c_build_command = c_compiler_path
c_build_multi_args = ["-std=c++17", "-Wall", "-g"] 
c_build_active_args = ["-std=c++17", "-Wall", "-g"]
# c_build_multi_args = ["-std=++c17", "-Wall", "-g", src_path, "-I", include_path, "-o", bin_path] 
# c_build_active_args = ["-std=++c17", "-Wall", "-g", "${file}", "-o", bin_path]
if os_name == "windows" and c_compiler_name == "cl.exe":
    c_build_multi_args = ["/std:c++17", "/nologo", "/Zi", "/EHsc"]
    c_build_active_args = ["/std:c++17", "/nologo", "/Zi", "/EHsc"]
    # c_build_multi_args = ["/std:c++17", "/nologo", "/Zi", "/EHsc", "/Fe:bin\\main.exe", "/Fo:bin\\", "/Fd:bin\\", "src\\*.cu", "/I", "include"]
    # c_build_active_args = ["/std:c++17", "/nologo", "/Zi", "/EHsc", "/Fe:bin\\main.exe", "/Fo:bin\\", "/Fd:bin\\", "${file}"]

c_compiler_dir = os.path.dirname(c_compiler_path)
xcompiler_multi_args = " ".join(c_build_multi_args)
xcompiler_active_args = " ".join(c_build_active_args)

cuda_build_command = cuda_compiler_path
cuda_build_multi_args = ["-Wno-deprecated-gpu-targets", "-g", "-G", src_path, "-I", include_path, "-o", bin_path, "--compiler-bindir", f'"{c_compiler_dir}"', "-Xcompiler", f'"{xcompiler_multi_args}"']
cuda_build_active_args = ["-Wno-deprecated-gpu-targets", "-g", "-G", "${file}", "-o", bin_path, "--compiler-bindir", f'"{c_compiler_dir}"', "-Xcompiler", f'"{xcompiler_active_args}"']
if os_name == "windows":
    cuda_build_multi_args = ["-Wno-deprecated-gpu-targets", "-g", "-G", src_path, "-I", include_path, "-o", bin_path]
    cuda_build_active_args = ["-Wno-deprecated-gpu-targets", "-g", "-G", "${file}", "-o", bin_path]

tasks_json = {
    "version": "2.0.0",
    "tasks": [
        {
            "type": "shell",
            "label": "Make directories",
            "command": makedir_command,
            "args": makedir_args,
            "problemMatcher": []
        },
        {
            "type": "shell",
            "label": "Clean .exe files",
            "dependsOn": ["Make directories"],
            "command": clean_command,
            "args": clean_args,
            "problemMatcher": []
        },
        {
            "type": "shell",
            "label": f"{cuda_compiler_name}: build multi file",
            "dependsOn": ["Clean .exe files"],
            "command": cuda_build_command,
            "args": cuda_build_multi_args,
            "options": {
                "cwd": "${workspaceFolder}"
            },
            "problemMatcher": [
                "$gcc", "$nvcc"
            ],
            "group": {
                "kind": "build",
                "isDefault": False
            },
            "detail": f"compiler: {cuda_compiler_path}"
        },
        {
            "type": "shell",
            "label": f"{cuda_compiler_name}: build active file",
            "dependsOn": ["Clean .exe files"],
            "command": cuda_build_command,
            "args": cuda_build_active_args,
            "options": {
                "cwd": "${fileDirname}"
            },
            "problemMatcher": [
                "$gcc", "$nvcc"
            ],
            "group": {
                "kind": "build",
                "isDefault": True
            },
            "detail": f"compiler: {cuda_compiler_path}"
        }
    ]
}

os.makedirs(".vscode", exist_ok=True)
json_string = json.dumps(tasks_json, indent=4)
with open(".vscode/tasks.json", "w") as f:
    json.dump(tasks_json, f, indent=4)

---
## 1.5 Create the File `launch.json`

- Run the cell below to create the file `launch.json` in subfolder `.vscode`.

In [5]:
import os, json

launch_json = {
    "version": "0.2.0",
    "configurations": [
        {
            "name": f"{cuda_debugger_name}: launch multi file",
            "preLaunchTask": f"{cuda_compiler_name}: build multi file",
            "type": "cuda-gdb",
            "request": "launch",
            "program": bin_path,
            "args": [],
            "stopAtEntry": False,
            "cwd": "${workspaceFolder}",
            "environment": []
        },
        {
            "name": f"{cuda_debugger_name}: launch active file",
            "preLaunchTask": f"{cuda_compiler_name}: build active file",
            "type": "cuda-gdb",
            "request": "launch",
            "program": bin_path,
            "args": [],
            "stopAtEntry": False,
            "cwd": "${workspaceFolder}",
            "environment": []
        }
    ]
}

os.makedirs(".vscode", exist_ok=True)
with open(".vscode/launch.json", "w") as f:
    json.dump(launch_json, f, indent=4)

---
## 1.6 Create the File `c_cpp_properties.json`

- Run the cell below to create the file `c_cpp_properties.json` in subfolder `.vscode`.

In [6]:
import os, json

if os_name == "linux" and c_compiler_name == "gcc":
    intelliSenseMode = "linux-gcc-x64"
    # intelliSenseMode = "linux-gcc-arm64"
if os_name == "linux" and c_compiler_name == "clang":
    intelliSenseMode = "linux-clang-x64"
    # intelliSenseMode = "linux-clang-arm64"
if os_name == "osx" and c_compiler_name == "gcc":
    intelliSenseMode = "macos-gcc-x64"
    # intelliSenseMode = "macos-gcc-arm64"
if os_name == "osx" and c_compiler_name == "clang":
    intelliSenseMode = "macos-clang-x64"
    # intelliSenseMode = "macos-clang-arm64"
if os_name == "windows" and c_compiler_name == "gcc.exe":
    intelliSenseMode = "windows-gcc-x64"
if os_name == "windows" and c_compiler_name == "clang.exe":
    intelliSenseMode = "windows-clang-x64"
if os_name == "windows" and c_compiler_name == "cl.exe":
    intelliSenseMode = "windows-msvc-x64"

launch_json = {
    "configurations": [
        {
            "name": "Linter",
            "includePath": [
                "${workspaceFolder}/**"
            ],
            "defines": [],
            "compilerPath": c_compiler_path,
            "cStandard": "c17",
            "cppStandard": "c++17",
            "intelliSenseMode": intelliSenseMode
        }
    ],
    "version": 4
}

os.makedirs(".vscode", exist_ok=True)
with open(".vscode/c_cpp_properties.json", "w") as f:
    json.dump(launch_json, f, indent=4)

---
## 1.7 VSCode Extensions

- To develop CUDA programs in C with VSCode, we need a few VSCode extensions (the last two are only needed for Jupyter Notebooks).
  - C/C++ Extension Pack: https://marketplace.visualstudio.com/items?itemName=ms-vscode.cpptools-extension-pack
  - CodeLLDB: https://marketplace.visualstudio.com/items?itemName=vadimcn.vscode-lldb
  - Makefile Tools: https://marketplace.visualstudio.com/items?itemName=ms-vscode.makefile-tools
  - Nsight Visual Studio Code Edition: https://marketplace.visualstudio.com/items?itemName=NVIDIA.nsight-vscode-edition
  - Jupyter: https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter
  - Python: https://marketplace.visualstudio.com/items?itemName=ms-python.python

- Run the cell below to install any missing VSCode extensions.

In [7]:
!code --install-extension ms-vscode.cpptools-extension-pack --force
!code --install-extension vadimcn.vscode-lldb --force
!code --install-extension ms-vscode.makefile-tools --force
!code --install-extension NVIDIA.nsight-vscode-edition --force
!code --install-extension ms-toolsai.jupyter --force
!code --install-extension ms-python.python --force

Installing extensions...
Extension 'ms-vscode.cpptools-extension-pack' is already installed.
Installing extensions...
Extension 'vadimcn.vscode-lldb' is already installed.
Installing extensions...
Extension 'ms-vscode.makefile-tools' is already installed.
Installing extensions...
Extension 'nvidia.nsight-vscode-edition' is already installed.
Installing extensions...
Extension 'ms-toolsai.jupyter' is already installed.
Installing extensions...
Extension 'ms-python.python' is already installed.


---
## 1.8 Using Notebook Extension `nvcc4jupyter` and Cell Magic `%%cuda`

- NVCC for Jupyter `nvcc4jupyter` is a Jupyter Notebook extension that let's us run CUDA code in notebook code cells.
  - It's installed into a Python environment with: `pip install nvcc4jupyter`
- It defines the cell magic `%%cuda` we can use at the beginning of a code cell, followed by CUDA code (in C or C++).
- For each cell with the cell magic `%%cuda`, `nvcc4jupyter` will store the source code and compiled executable in a temporary folder.
  - Below, we are remapping the temporary folder to the folder `./nvcc4jupyter` (so we can see what files are created).
  - For each cell with the cell magic `%%cuda`, a randomly named subfolder will be created under `./nvcc4jupyter`, containing two files:
    - `single_file.cu` which is the source code copied from the notebook cell.
    - `cuda_exec.out` which is the compiled executable.

### Create Folder for `nvcc4jupyter`

- Run the cell below to create the folder `nvcc4jupyter` in the workspace folder.

In [8]:
import os
import shutil
import tempfile

_fixed_temp_dir = os.path.abspath("./nvcc4jupyter")
if os.path.exists(_fixed_temp_dir):
    shutil.rmtree(_fixed_temp_dir)
os.makedirs(_fixed_temp_dir, exist_ok=True)
tempfile.gettempdir = lambda: _fixed_temp_dir

### Load `nvcc4jupyter` Notebook Extension

- Jupyter Notebook extensions need to be loaded into the notebook with the cell magic `%load_ext` (or reloaded with `%reload_ext`).
- Run the cell below to load the `nvcc4jupyter` extension.

In [9]:
%load_ext nvcc4jupyter
%reload_ext nvcc4jupyter

Source files will be saved in "/home/patrick/projects/cuda5/nvcc4jupyter/tmp5838f4ri".
Source files will be saved in "/home/patrick/projects/cuda5/nvcc4jupyter/tmpdr_qpxmp".


### Use Cell Magic `%%cuda`

- Run the cell below, containing a simple CUDA program written in C.
  - Notice the code cell starts with the cell magic `%%cuda`, followed by a CUDA program written in C.
  - The program uses `printf()` to print out text from both `host` code (runs on the CPU) and `device` code (runs on the GPU).
- Then inspect the folder `./nvcc4jupyter`.
  - It contains a subfolder with a random name, which contains two files:
    - `single_file.cu` which contains the CUDA source code, i.e. code cell's contents (except for the cell magic command `%%cuda`).
    - `cuda_exec.out` which is the executable file, compiled from the source code by NVCC.

In [10]:
%%cuda
#include <stdio.h>
#include <cuda_runtime.h>

__global__ void hello()
{
    // CUDA supports the printf() function on the device
    printf("Hello from device -> block: %u, thread: %u\n", blockIdx.x, threadIdx.x);
}

int main()
{
    printf("Hello from host\n");
    hello<<<2, 2>>>();
    cudaDeviceSynchronize();
    return 0;
}

Hello from host
Hello from device -> block: 1, thread: 0
Hello from device -> block: 1, thread: 1
Hello from device -> block: 0, thread: 0
Hello from device -> block: 0, thread: 1



---
## 1.9 Using Built-in Cell Magic `%%writefile`

- The cell magic `%%writefile filename`, writes the contents of a notebook cell to the specified `filename` (or file path).
  - This functionality is built-in to Jupyter Notebooks (it's not an extension).
  - We can use it to write any code cell contents to a file in the file system.
  - Let's write some CUDA code to the file `main.cu`.
- Run the cell below.
- Then inspect the resulting file `main.cu`, which contains the code cell's contents (except for the cell magic command `%%writefile filename`).

In [11]:
%%writefile main.cu
#include <stdio.h>
#include <cuda_runtime.h>

__global__ void hello()
{
    // CUDA supports the printf() function on the device
    printf("Hello from device -> block: %u, thread: %u\n", blockIdx.x, threadIdx.x);
}

int main()
{
    printf("Hello from host\n");
    hello<<<2, 2>>>();
    cudaDeviceSynchronize();
    return 0;
}

Writing main.cu


---
## 1.10 Compiling and Executing a CUDA Program From a Notebook Code Cell

- We can compile and execute a CUDA program from a notebook code cell using the syntax `!<shell command>`, where:
  - `!` indicates that the succeeding text on the same row should be sent to the shell (terminal).
  - `<shell command>` is the shell (terminal) command we want to execute.
  - Standard output is redirected to the cell output.

- Run the cell below to see what the build command and execute command is in your shell:
  - The *build single file* command compiles and links the file `main.cu` in your workspace folder and places the executable file `main.exe` in the `bin` folder.
  - The *build multi file* command compiles and links all `.cu` files in the `src` and `.h` files in the `include` folder and places the executable file `main.exe` in the `bin` folder.
  - The *execute* command executes the file `main.exe` in the `bin` folder.

In [12]:
#cuda_build_command = f'"{cuda_build_command}"'
build_single_file_command = [f'"{cuda_build_command}"'] + cuda_build_active_args
build_single_file_command = " ".join(build_single_file_command).replace('${file}', 'main.cu').replace('${workspaceFolder}', '.')

build_multi_file_command = [f'"{cuda_build_command}"'] + cuda_build_multi_args
build_multi_file_command = " ".join(build_multi_file_command).replace('${file}', 'main.cu').replace('${workspaceFolder}', '.')

execute_command = bin_path.replace('${workspaceFolder}', '.')

print(f'Build single file command : {build_single_file_command}')
print(f'Build multi file command  : {build_multi_file_command}')
print(f'Execute command           : {execute_command}')

Build single file command : "/usr/local/cuda/bin/nvcc" -Wno-deprecated-gpu-targets -g -G main.cu -o ./bin/main.exe --compiler-bindir "/usr/bin" -Xcompiler "-std=c++17 -Wall -g"
Build multi file command  : "/usr/local/cuda/bin/nvcc" -Wno-deprecated-gpu-targets -g -G ./src/*.cu -I ./include -o ./bin/main.exe --compiler-bindir "/usr/bin" -Xcompiler "-std=c++17 -Wall -g"
Execute command           : ./bin/main.exe


- Run the cell below to:
  - Create the folder `bin` in your workspace folder if it doesn't already exist.
  - Build the single source code file `main.cu` in your workspace foilder into the executable file `main.exe` in the `bin` folder.
  - Run the executable file `main.exe` in the `bin` folder.
- Notice the file `main.exe` has been created in the file system (in the `bin` folder), and the program's output is shown as the cell's output in the notebook. 

In [13]:
import os
os.makedirs("bin", exist_ok=True)

!{build_single_file_command}
!{execute_command}

Hello from host
Hello from device -> block: 1, thread: 0
Hello from device -> block: 1, thread: 1
Hello from device -> block: 0, thread: 0
Hello from device -> block: 0, thread: 1


---
## 1.11 Compiling and Debugging a Single-file CUDA Program

Let's see `tasks.json` and `launch.json` in action for a single-file (`.cu`) C program.

- First, let's create the file `main.cu` in the cell below.

In [14]:
%%writefile main.cu
#include <stdio.h>
#include <cuda_runtime.h>

__global__ void hello()
{
    // CUDA supports the printf() function on the device
    printf("Hello from device -> block: %u, thread: %u\n", blockIdx.x, threadIdx.x);
}

int main()
{
    printf("Hello from host\n");
    hello<<<2, 2>>>();
    cudaDeviceSynchronize();
    return 0;
}


Overwriting main.cu


**NOTE! YOU NEED TO BE USING GCC AS YOUR COMPILER ON LINUX TO DEBUG CUDA PROGRAMS IN VSCODE!**

- Now, let's debug the file `main.cu`.
  - Open the file `main.cu` in VSCode's editor.
  - Set a breakpoint on the two `printf()` statements (`F9`).
  - Switch to the `Run and Debug` view (Linux/Windows: `Ctrl + Shift + D`, Mac: `Cmd + Shift + D`).
  - In the drop-down combobox, select the launch configuration `cuda-dbg: launch active file`.
  - Click the green `Play` icon.
  - Use the debug toolbar in the top-middle of VSCode to debug the code.
    - Notice the debugger stops at the two breakpoints (both in the host code and the device code).
      - This is because we are using `cuda-dbg`as the debugger.
    - Notice you can view variables (local, registers), watch variables, view the call stack, and toggle breakpoints in the `Run and Debug` view.
  - Stop debugging (red `Square` icon in the debug toolbar).
- Now look at the status bar (at the bottom of VSCode) where you will see the name of the launch configuration `cuda-gdb: launch active file`.
  - Click on it, and select `cuda-gdb: launch active file` again (make sure `main.cu` is the active file in the editor, not the notebook).
    - This is an alternative method to start a debug session.
  - Stop debugging.
- Press `F5` (make sure `main.cu` is active in VSCode's editor), which is a third alternative to launch the debugger.
  - This launches the debug configuration with `preLaunchTask` set to the default task (in `tasks.json`).
  - Stop debugging.
- Press (Linux/Windows: `Ctrl + Shift + B`, Mac: `Ctrl + Shift + B`) to execute the default build task (in `tasks.json`).
  - Make sure `main.cu` is active in VSCode's editor (since the default build task is set to the active file task).
  - Notice the compiled executable `main.exe` is placed in the subfolder `bin` (configured in the default build task).
    - This is also where the debugger finds the executable `main.exe` (configured in `launch.json`).

- Remeber, you can always compile a single `main.cu` file in your workspace folder and run it using the commands below.

In [15]:
!{build_single_file_command}
!{execute_command}

Hello from host
Hello from device -> block: 1, thread: 0
Hello from device -> block: 1, thread: 1
Hello from device -> block: 0, thread: 0
Hello from device -> block: 0, thread: 1


---
## 1.12 Compiling and Debugging a Multi-file CUDA Program

- Let's see `tasks.json` and `launch.json` in action for a multi-file (`*.cu`) CUDA program.
  - First We will create two source code files `.cu` in the `src` folder, and one header file `.h` in the `include` folder.
    - We will use the same code as before, but will place the CUDA kernel (function) code in its own `.cu` file and its prototype in a header file `.h`.
  - Then we will use:
    - The other (non-default) build task in `tasks.json` to build the executable.
    - The other launch configuration (linked to the non-default build task) in `launch.json` to debug it.

- Run the four cells below to create:
  - The folder structure `src`, `include`, and `bin` (if it hasn't already been created).
  - The main source code file `main.cu` in the folder `src`.
  - The kernel source code file `helloKernel.cu` in the folder `src`.
  - The kernel prototype in the header file `helloKernel.h` in the folder `include`.

In [16]:
import os
os.makedirs("src", exist_ok=True)
os.makedirs("include", exist_ok=True)
os.makedirs("bin", exist_ok=True)

In [17]:
%%writefile src/main.cu
#include <stdio.h>
#include <cuda_runtime.h>
#include "helloKernel.h"

int main()
{
    printf("Hello from host\n");
    hello<<<2, 2>>>();
    cudaDeviceSynchronize();
    return 0;
}

Writing src/main.cu


In [18]:
%%writefile src/helloKernel.cu
#include <stdio.h>
#include <cuda_runtime.h>
#include "helloKernel.h"

__global__ void hello()
{
    // CUDA supports the printf() function on the device
    printf("Hello from device -> block: %u, thread: %u\n", blockIdx.x, threadIdx.x);
}

Writing src/helloKernel.cu


In [19]:
%%writefile include/helloKernel.h
#pragma once
#include <cuda_runtime.h>

__global__ void hello();

Writing include/helloKernel.h


**NOTE! YOU NEED TO BE USING GCC AS YOUR COMPILER ON LINUX TO DEBUG CUDA PROGRAMS IN VSCODE!**

- Build the multi-file CUDA program.
  - Notice the multi-file build task in `tasks.json` isn't the default build task (`isDefault` is not set to `true` under `group`).
  - Therefore, we can't use (Linux/Windows: `Ctrl + Shift + B`, mac: `Cmd + Shift + B`).
  - Instead we can:
    - Bring up the Command Palette (Linux/Windows: `Ctrl + Shift + P`, mac: `Cmd + Shift + P`).
    - Choose `Tasks: Run Task` and select the task `nvcc: build multi file`.
  - The executable `main.exe` is placed in the `bin` folder (as configured in `tasks.json`).
- Debug the multi-file CUDA program.
  - Open `main.cu` and `addKernel.cu` in the `src` folder and set breakpoints on the two `printf()` statements.
  - Notice the multi-file launch task in `launch.json` isn't linked to the default build task (in `tasks.json`).
    - Therefore, we can't use `F5`.
    - Instead we can:
      - Switch the the `Run and Debug` view, select `cuda-dbg: launch multi file` from the drop-down list, and click the green `Play` icon.
      - Or select `cuda-dbg: launch multi file` from the status bar (at the bottom of VSCode).
    - The CUDA program is built and the debugger lauched, attaching to the executable `main.exe` in the `bin` folder.
  - Stop debugging.
- Remeber, you can always compile a multi-file CUDA program (`src/*.cu`, `include/*.h`) and run it using the commands below.

In [20]:
!{build_multi_file_command}
!{execute_command}

Hello from host
Hello from device -> block: 1, thread: 0
Hello from device -> block: 1, thread: 1
Hello from device -> block: 0, thread: 0
Hello from device -> block: 0, thread: 1


---
## 1.13 Using NVIDIA NSight Compute (`ncu`)

NVIDIA Nsight Compute (`ncu`) is a CUDA kernel profiler (command-line tool) provided by NVIDIA. It's used to analyze and optimize the performance of CUDA applications by collecting low-level performance metrics from the GPU.

- NVidia's documentation is a good source for detailed information about its tools:
  - [Nsight Compute CLI](https://docs.nvidia.com/nsight-compute/NsightComputeCli/index.html)
  - [Nsight Developer Tools](https://docs.nvidia.com/nsight-developer-tools/index.html)
- In NVIDIA Nsight Compute, there is:
  - A command-line tool : `ncu`
  - An application with a graphical user interface: `ncu-ui`
    - On Windows, open the start menu and search for `Nsight Compute`.
- To profile our executable file `main.exe` in the folder `bin` using `ncu`, we can run the command:
  
  ```bash
  ncu bin/main.exe
  ```
- To create a profile report and open it in the graphical user interface, we can run the commands below:
  - Note that compiling the source code with the flag `-lineinfo` makes line number available in `ncu-ui`.

  ```bash
  nvcc -lineinfo -o bin/main.exe main.cu
  ncu -o profile_report bin/main.exe
  ncu-ui profile_report.ncu-rep
  ```
- Let's try the `ncu` tool.
  - Open a terminal and run the command below
    
    ```bash
    ncu bin/main.exe
    ```

**Note**

- If you get the message: `The user does not have permission to access NVIDIA GPU Performance Counters on the target device 0. For instructions on enabling permissions and to get more information see https://developer.nvidia.com/ERR_NVGPUCTRPERM`.
- This means you need to run `ncu` as an administrator on your computer
- See: https://developer.nvidia.com/nvidia-development-tools-solutions-err_nvgpuctrperm-permission-issue-performance-counters
  - On Windows, you need to run `ncu` from a terminal with administrative rights.
  - On Linux and Mac, you need to use `sudo ncu`.
  - Depending on your Linux distribution, you can also enable anyone to use `ncu` without `sudo ncu`, e.g. for Ubuntu (or other Debian-based ditributions):
      
    ```bash
    echo "options nvidia NVreg_RestrictProfilingToAdminUsers=0" | sudo tee /etc/modprobe.d/nvidia-profiler.conf
    sudo update-initramfs -u
    sudo reboot
    ```
- You can run `ncu` in a notebook cell, only if you have administrator rights (Windows), or don't have to use `sudo ncu` (Linux/Mac).
  - Otherwise the notebook cell will "hang", waiting on authorization (a password in Linux).

<br />
<br />

**The output from `ncu` will look something like below which contains important information regarding optimizing CUDA code ...**

```bash
==PROF== Disconnected from process 12856
[12856] main.exe@127.0.0.1
  hello() (2, 1, 1)x(2, 1, 1), Context 1, Stream 7, Device 0, CC 8.9
    Section: GPU Speed Of Light Throughput
    ----------------------- ------------- ------------
    Metric Name               Metric Unit Metric Value
    ----------------------- ------------- ------------
    DRAM Frequency          cycle/nsecond         6.51
    SM Frequency            cycle/usecond       866.15
    Elapsed Cycles                  cycle       93,159
    Memory Throughput                   %         0.29
    DRAM Throughput                     %         0.00
    Duration                      usecond       107.55
    L1/TEX Cache Throughput             %         2.37
    L2 Cache Throughput                 %         0.29
    SM Active Cycles                cycle     7,646.04
    Compute (SM) Throughput             %         0.19
    ----------------------- ------------- ------------

    WRN   This kernel grid is too small to fill the available resources on this device, resulting in only 0.0 full      
          waves across all SMs. Look at Launch Statistics for more details.                                             

    Section: Launch Statistics
    -------------------------------- --------------- ---------------
    Metric Name                          Metric Unit    Metric Value
    -------------------------------- --------------- ---------------
    Block Size                                                     2
    Function Cache Configuration                     CachePreferNone
    Grid Size                                                      2
    Registers Per Thread             register/thread              32
    Shared Memory Configuration Size           Kbyte           32.77
    Driver Shared Memory Per Block       Kbyte/block            1.02
    Dynamic Shared Memory Per Block       byte/block               0
    Static Shared Memory Per Block        byte/block               0
    Threads                                   thread               4
    Waves Per SM                                                0.00
    -------------------------------- --------------- ---------------

    WRN   Threads are executed in groups of 32 threads called warps. This kernel launch is configured to execute 2      
          threads per block. Consequently, some threads in a warp are masked off and those hardware resources are       
          unused. Try changing the number of threads per block to be a multiple of 32 threads. Between 128 and 256      
          threads per block is a good initial range for experimentation. Use smaller thread blocks rather than one      
          large thread block per multiprocessor if latency affects performance.  This is particularly beneficial to     
          kernels that frequently call __syncthreads(). See the Hardware Model                                          
          (https://docs.nvidia.com/nsight-compute/ProfilingGuide/index.html#metrics-hw-model) description for more      
          details on launch configurations.                                                                             
    ----- --------------------------------------------------------------------------------------------------------------
    WRN   The grid for this launch is configured to execute only 2 blocks, which is less than the GPU's 24              
          multiprocessors. This can underutilize some multiprocessors. If you do not intend to execute this kernel      
          concurrently with other workloads, consider reducing the block size to have at least one block per            
          multiprocessor or increase the size of the grid to fully utilize the available hardware resources. See the    
          Hardware Model (https://docs.nvidia.com/nsight-compute/ProfilingGuide/index.html#metrics-hw-model)            
          description for more details on launch configurations.                                                        

    Section: Occupancy
    ------------------------------- ----------- ------------
    Metric Name                     Metric Unit Metric Value
    ------------------------------- ----------- ------------
    Block Limit SM                        block           24
    Block Limit Registers                 block           64
    Block Limit Shared Mem                block           32
    Block Limit Warps                     block           48
    Theoretical Active Warps per SM        warp           24
    Theoretical Occupancy                     %           50
    Achieved Occupancy                        %         2.08
    Achieved Active Warps Per SM           warp            1
    ------------------------------- ----------- ------------

    WRN   This kernel's theoretical occupancy (50.0%) is limited by the number of blocks that can fit on the SM. The    
          difference between calculated theoretical (50.0%) and measured achieved occupancy (2.1%) can be the result    
          of warp scheduling overheads or workload imbalances during the kernel execution. Load imbalances can occur    
          between warps within a block as well as across blocks of the same kernel. See the CUDA Best Practices Guide   
          (https://docs.nvidia.com/cuda/cuda-c-best-practices-guide/index.html#occupancy) for more details on           
          optimizing occupancy.
```

---
# 2. CUDA Basics
---

- Now we know how to create CUDA programs in C (single-file, multi-file, and in a Jupyter Notebook cell).
- Going forward, we will explore fundamental CUDA programming concepts as single-file programs in notebook cells using the `%%cuda` cell magic command.
  - The code in each `%%cuda` cell can be placed in a `main.cu` file by:
    - Manually copying the cell contents and removing the `%%cuda` row.
    - Replacing the `%%cuda` row with `%%writefile main.cu` and running the cell.
  - Then you can manually compile it to `main.exe` with `nvcc main.cu -o main.exe`.
- NVidia's documentation is a good source to learn more about CUDA:
    - [Cuda C Programming Guide](https://docs.nvidia.com/cuda/cuda-c-programming-guide/contents.html)

---
## 2.1 Listing CUDA-enabled Devices and Properties

- First, let's find out what CUDA-enabled devices are available on your computer.

### Using `nvidia-smi`

- The simplest way to list CUDA-enabled devices is using the tool `nvidia-smi`.
  - The top row shows you:
    - The `nvidia-smi version` (left).
    - The NVidia GPU `driver version` (middle).
    - The supported `CUDA runtime version` (right).
  - It also shows you:
    - Information about your CUDA-enabled devices (beneath the top row).
    - What processes are running on your CUDA-enabled devices (bottom).
- Run the cell below to see the CUDA-enabled devices on your system.

In [21]:
!nvidia-smi

Mon Jul 14 22:42:35 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.124.06             Driver Version: 570.124.06     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX 2000 Ada Gene...    Off |   00000000:01:00.0 Off |                  N/A |
| N/A   51C    P3             10W /   35W |       1MiB /   8188MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

- Let's also check the NVidia Cuda Compiler (NVCC) version by running the cell below.
  - Notice the `nvcc version` in the two last rows (release and build).
  - Now compare the `nvcc version` with the supported `CUDA runtime version` above (top right output from `nvidia-smi`).
    - The `CUDA runtime version` is the **maximum** supported CUDA runtime version.
    - The `nvcc version` is the same are the CUDA Toolkit (SDK) version.
    - **The `nvcc version` must be less than or equal to the `CUDA runtime version`**.
      - Otherwise your CUDA programs in C won't work.

In [22]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2025 NVIDIA Corporation
Built on Fri_Feb_21_20:23:50_PST_2025
Cuda compilation tools, release 12.8, V12.8.93
Build cuda_12.8.r12.8/compiler.35583870_0


### Using C Code

- We can also find out what CUDA-enabled devices are available using C code.
  - The code below lists important properties that will become familiar the more you learn about CUDA (for optimization purposes).
- Run the cell below to list CUDA-enabled devices and their properties.

In [23]:
%%cuda
#include <stdio.h>
#include <cuda_runtime.h>

int _ConvertSMVer2Cores(int major, int minor)
{
    typedef struct { int SM; int Cores; } SMtoCores;
    SMtoCores map[] = {
        {0x30, 192}, {0x32, 192}, {0x35, 192}, {0x37, 192},  // Kepler
        {0x50, 128}, {0x52, 128}, {0x53, 128},               // Maxwell
        {0x60,  64}, {0x61, 128}, {0x62, 128},               // Pascal
        {0x70,  64}, {0x72,  64}, {0x75,  64},               // Volta/Turing
        {0x80,  64}, {0x86, 128},                            // Ampere
        {0x89, 128}, {0x90, 128},                            // Ada/Hopper
        {-1, -1}
    };

    int sm = ((major << 4) + minor);
    for (int i = 0; map[i].SM != -1; ++i)
    {
        if (map[i].SM == sm)
            return map[i].Cores;
    }
    return -1;  // Unknown
}

int main()
{
    int deviceCount;
    cudaGetDeviceCount(&deviceCount);

    puts("Nvidia Devices:\n");
    for (int i = 0; i < deviceCount; i++)
    {
        cudaDeviceProp prop;
        cudaGetDeviceProperties(&prop, i);

        printf("- Device %d: %s\n", i, prop.name);
        printf("  - %-31s : %d.%d\n", "Compute capability", prop.major, prop.minor);
        printf("  - %-31s : %d\n", "Streaming Multiprocessors (SMs)", prop.multiProcessorCount);
        printf("  - %-31s : %d\n", "Streaming Processors (SPs)", prop.multiProcessorCount * _ConvertSMVer2Cores(prop.major, prop.minor));
        printf("  - %-31s : %d\n", "Max threads per SM", prop.maxThreadsPerMultiProcessor);
        printf("  - %-31s : %d\n", "Max threads per block", prop.maxThreadsPerBlock);
        printf("  - %-31s : %d\n", "Warp size", prop.warpSize);
        printf("  - %-31s : %d\n", "Registers per block", prop.regsPerBlock);
        printf("  - %-31s : %lu bytes\n", "Register file size per SM", prop.regsPerBlock * 4);
        printf("  - %-31s : %lu bytes\n", "Shared memory per SM", prop.sharedMemPerMultiprocessor);
        printf("  - %-31s : %lu bytes\n", "Shared memory per block", prop.sharedMemPerBlock);
        printf("  - %-31s : %lu bytes\n", "Constant memory size", prop.totalConstMem);
        printf("  - %-31s : %lu bytes\n", "Global memory size", prop.totalGlobalMem);
        printf("  - %-31s : %.2f MHz\n", "Clock rate", prop.clockRate / 1000.0);
        printf("\n");
    }

    // Sets device 0 as the current device
    //cudaSetDevice(0);

    return 0;
}

Nvidia Devices:

- Device 0: NVIDIA RTX 2000 Ada Generation Laptop GPU
  - Compute capability              : 8.9
  - Streaming Multiprocessors (SMs) : 24
  - Streaming Processors (SPs)      : 3072
  - Max threads per SM              : 1536
  - Max threads per block           : 1024
  - Warp size                       : 32
  - Registers per block             : 65536
  - Register file size per SM       : 262144 bytes
  - Shared memory per SM            : 102400 bytes
  - Shared memory per block         : 49152 bytes
  - Constant memory size            : 65536 bytes
  - Global memory size              : 8198619136 bytes
  - Clock rate                      : 1455.00 MHz




---
## 2.2 Hello World in Host Code (CPU)

- The program in the cell below is a simple C program (no CUDA code) that runs on the host (CPU).
  - We include the necessary header files:
    - `stdio.h` for `printf`.

      ```c
      #include <stdio.h>
      ```
  - Then we define the `main()` function:
    - We print out the text `Hello World!`.
    - Then we return the exit code `0` to the operating system.
    
      ```c
      int main(void)
      {
        printf("Hello World!\n");
        
        return 0;
      }
      ```
- Run the cell below to see the output.

In [24]:
%%cuda
#include <stdio.h>

// Host entry point (a normal C main function)
int main(void)
{
   printf("Hello World!\n");
   
   return 0;
}

Hello World!



---
## 2.3 Hello World in Device Code (GPU)

- The program in the cell below is a simple CUDA program that runs code on the host (CPU) and the device (GPU).
  - We include the necessary header files:
    - `stdio.h` for `printf`.
    - `cuda_runtime.h` for `cudaDeviceSynchronize`.

      ```c
      #include <stdio.h>
      #include <cuda_runtime.h> // contains CUDA function prototypes
      ```
  - Then we define a CUDA kernel function:
    - `mykernel()` is the name of the **kernel function** (it can be any name we like).
    - A **kernel function** runs on the device (GPU) and is **launched** (called) from the host (CPU).
    - `__global__` is a CUDA **qualifier** (qualifying a function with `__global__` makes the function a **kernel function**.
    - The kernel function takes no arguments (`void`), prints the text `Hello World!`, and does NOT return a value (`void`).
    - All **kernel functions** MUST have a `void` return type.

      ```c
      __global__ void mykernel(void) // the __global__ qualifier makes this a kernel function
      {
          printf("Hello World!\n");
      }
      ```
  - Lastly we define the `main()` function:
    - First we **launch** (call) the **kernel function** `mykernel()` using the syntax `kernel<<<n_blocks, n_threads>>>()`.
      - `kernel` is the name of the **kernel function** (`mykernel` in our code), which is called from the host (CPU) and runs on the device (GPU).
      - `<<<n_blocks, n_threads>>>` is a CUDA **launch configuration** (`<<<1, 1>>>` in our code), which makes this a **kernel launch** (kernel function call).
      - `n_blocks` is the number of `blocks` to use when launching the kernel function.
        - In Nvidia terminology, a CUDA-enabled GPU contains a `grid` of `blocks`, where each `block` contains a number of `threads`.
      - `n_threads` is the number of `threads` to use when launching the kernel function.
        - Each `block` contains this number of `threads`.
      - The code `mykernel<<<1,1>>>()` launches the kernel (calls the function) `mykernel()` on the GPU with 1 `block` containing 1 `thread` in that `block`.
        - It's the *equivalent* of creating 1 new thread and running the function on that thread in a traditional C program running on the host (CPU).
        - The kernel launch is an asynchronous function call, so it immediately returns control to the host (CPU).
    - Then we call the CUDA function `cudaDeviceSynchronize()`.
      - This function call is a synchronous call, which blocks the host's (CPU's) main thread until the kernel function completes (returns) on the device (GPU).
      - This is necessary, otherwise the `main()` function would go out of scope (terminate) before we have a chance to retrieve any results from the kernel function.
    - Finally we return the exit code `0` to the operating system.
    
      ```c
      int main(void)
      {
        mykernel<<<1, 1>>>();    // kernel launch (calls the kernel function mykernel, and is asynchronous)
        cudaDeviceSynchronize(); // blocks the host's main thread (CPU) until the kernel function completes (returns)
        
        return 0;
      }
      ```
- Run the cell below to see the output.

**TL;DR**

- The CUDA function prototypes are declared in header file `cuda_runtime.h`.
- A function with the `__global__` qualifier is called a **kernel function** that **runs on the device (GPU)** and is **called from the host (CPU)**.
- A **kernel launch** calls a **kernel function** using the syntax `kernelfunction<<<n_blocks, n_threads>>>()` and is an **asynchrounous call**.
- The function `cudaDeviceSynchronize()` is a **synchronous** call that blocks the host code until the **kernel function** is complete (returns).
- The Nvidia compiler `nvcc` separates source code into **host** and **device** components.
  - Device functions (e.g. `mykernel()`) is processed by the NVIDIA compiler `nvcc`.
  - Host functions (e.g. `main()`) are processed by a standard host C compiler (e.g. `gcc`, `clang`, or `cl.exe`).
    - NVCC instructs the underlying C compiler to compile host code during the compilation process.

In [25]:
%%cuda
#include <stdio.h>
#include <cuda_runtime.h>

__global__ void mykernel(void)
{
    printf("Hello World!\n");
}

int main(void)
{
   mykernel<<<1, 1>>>();
   cudaDeviceSynchronize();
   
   return 0;
}

Hello World!



---
## 2.4 Grids, Blocks, Threads, Devices, SMs, and SPs

- In the previous CUDA code, we saw:
  - A CUDA **kernel**, where:
    - `__global__` is a CUDA-specific qualifier that makes a function a **kernel function**.
      - A **kernel function** is executed on the host (GPU), and called from the host (GPU).
    - `kernel` is the name of the **kernel function**.
    - `parameterlist` is the kernel function's comma-separated list of parameters.

      ```c
      __global__ kernel(parameterlist)
      {

      }
      ```
  - A CUDA **kernel launch**, where:
    - `kernel` is the name of the **kernel** function.
    - `<<<  >>>` is a special syntax used to **launch** (call) a **kernel function** and contains **launch parameters**.
    - `blocks` is a **launch parameter** with the number of **blocks per grid**.
    - `threads` is a **launch parameter** with the number of **threads per block**.
    - `argumentlist` are the arguments passed to the kernel function (and must match its parameterlist above).
  
      ```bash
      kernel<<<blocks, threads>>>(argumentlist);
      ```

<img src="images/cuda.png" width="500" style="float: right; margin-right: 50px;" />

- A CUDA-enabled GPU:
  - Is called a `Device`.
  - Has a number of `Streaming Multiprocessors (SMs)`.
  - Each `SM` has a number of `Streaming Processors (SPs)`.
- During a **kernel launch** `kernel<<<blocks, threads>>>(argumentlist)`:
  - CUDA lauches a `grid`. 
  - A `grid` contains a number of `blocks` (specified as `blocks` within `<<<blocks, threads>>>`).
  - Each `block` contains a number of `threads` (specified as `threads` within `<<<blocks, threads>>>`).
- CUDA maps:
  - A `block` to run on an `SM` (multiple `blocks` can be assigned to the same `SM`).
  - Each `thread` within a `block` to an `SP`.
- So, a `block` runs on an `SM`, and each `thread` within that `block` runs on a `SP` within that `SM`.

- For each **kernel launch**:
  - CUDA creates a `grid` containing `blocks` containing `threads`.
  - Each `thread` runs a copy of the same `kernel` function with the same `argumentlist`.
  - 4 CUDA-specific global variables are available within each copy of the `kernel` function:
    - `gridDim` of type `dim3`, which is a `struct` with three member variables `int x`, `int y`, and `int z`.
      - This is the number of `blocks` in a `grid`, and can be specified as a 1D (`x`), 2D (`y`), or 3D (`z`) set of `blocks`.
    - `blockDim` of type `dim3`, which is a `struct` with three member variables `int x`, `int y`, and `int z`.
      - This is the number of `threads` in a `block`, and can be specified as a 1D (`x`), 2D (`y`), or 3D (`z`) set of `threads`.
    - `blockIdx` of type `dim3`, which is a `struct` with three member variables `int x`, `int y`, and `int z`.
      - This is unique ID of a `block` within the `grid`, and has a 1D (`x`), 2D (`y`), and 3D (`z`) ID.
    - `threadIdx` of type `dim3`, which is a `struct` with three member variables `int x`, `int y`, and `int z`.
      - This is unique ID of a `thread` within a `block`, and has a 1D (`x`), 2D (`y`), and 3D (`z`) ID.

      ```c
      typedef struct
      {
        int x;
        int y;
        int z;
      } dim3;
      ```
    
  - In the figure, we see that each `SM` contains:
    - A `register file` (the blue rectangle).
      - The `register file` is divided into `**chunks**, where each `thread` (running in the `block` assigned to the `SM`) is assigned one **chunk**.
      - Each `thread`'s chunk of the `register file` is referred to as the `thread`'s `private memory` (only that `thread` can access that `private memory`).
      - In a `kernel` function, we can declare a local variable with the qualifier `__private__` which stores that variable in a `thread`'s `private memory`.
        - If we don't specify a qualifier, the variable is by default stored in the `thread`'s `private memory`.
    - A `shared memory` buffer (the green rectangle).
      - The `shared memory` is shared by all `threads` running in the `block` assigned to that `SM`.
      - To store a variable in `shared memory`, we declare the variable with the `__shared__` qualifier.
  - There is also `global memory`, which is declared using the qualifier `__global__`.
    - If we don't explicitly qualify a parameter in the kernel function's parameterlist with a qualifier, it is by default `__global__`, i.e. referring to `global memory`.
  - Finally, there is `constant memory` stored outside an `SM` (just like `global memory`), and is `read-only` memory (it can we written to once before a kernel launch).
    - `constant memory` is small, but very effient (the CUDA compiler can optimize access to it since it is `read-only`).

**TL;DR**
- A CUDA-enabled GPU is referred to as a `device`, and has an array of `SMs`, each with a number of `SPs`.
- A kernel launch specifies the number of `blocks` and `threads` to use within `<<<blocks, threads>>>`.
- CUDA maps a `block` to an `SM`.
- Each `thread` within a `block` is run on an `SP` within that `SM`.
- A CUDA-enabled GPU has `global memory` and `constant memory` located outside of any `SM`.
- Each `SM` has `shared memory` (shared by all `threads` running in the `block` assigned to an `SM`).
- Each `SM` has a `register file`, divided into chunks, where each `thread` is a assigned a chunk referred to as `private memory` (private to that `thread`).
- Each `thread` runs a **copy** of the same `kernel` function with the same `argumentlist`.

### Code Demonstrating Grids, Blocks, and Threads

- Let's write a CUDA program that demonstrates running a `grid` of `blocks`, each with a number of `threads`, on the device (GPU).
- The program copies the elements from one 1D `int` array `input` to another 1D `int` array `output`.
  - We include the necessary header files:
    - `stdio.h` for `printf`
    - `stdlib.h` for `malloc` and `free`
    - `time.h` for `srand` and `rand`
    - `cuda_runtime.h` for `cudaMalloc`, `cudaMemcpy`, and `cudaFree`
    
    ```c
    #include <stdio.h>
    #include <stdlib.h>
    #include <time.h>
    #include <cuda_runtime.h>
    ```
  
  - We define symbolic constants:
    - `N` with the value `5` for the number of elements in each array
    - `THREADS_PER_BLOCK_X` with the value `2` for the number of `threads` in each `block`

    ```c
    #define N 5
    #define THREADS_PER_BLOCK_X 2
    ```
  
  - We define a kernel function called `kernel`.
    - It has the `__global__` qualifier, making it a kernel function, returns `void`, and has three parameters:
      - An `int *` pointer `input` which points to the `input` array in the device's `global memory`.
      - An `int *` pointer `output` which points to the `output` array in the device's `global memory`.
      - An `int` variable `n` with the number of elements in each array.
    - In the function's body, we:
      - Use the CUDA-specific global variables `gridDim`, `blockDim`, `blockIdx`, and `threadIdx`, all of type `dim3` (with `int` member variables `x`, `y`, `z`).
        - CUDA sets these as global variables, behind the scenes, during a kernel launch.
        - They are available to each thread within each copy of the kernel function.
        - `gridDim.x` is the number of `blocks` in a `grid`.
          - In our case we have `(5 + 2 - 1) / 2 = 3` since we calculate it as `(N + THREADS_PER_BLOCK_X - 1) / THREADS_PER_BLOCK_X` in the `main` function.
        - `blockDim.x` is the number of `threads` in a `block`.
          - In our case we have `2`, defined by `THREADS_PER_BLOCK_X`.
        - `blockIdx.x` is the unique `ID` of a `block` within the `grid`.
          - Since `gridDim.x` is `3`, `blockIdx.x` ranges from `0` to `2` (zero-based, i.e. from `0` to `gridDim.x - 1`).
        - `threadIdx.x` is the unique `ID` of a `thread` within a `block`.
          - Since `blockDim.x` is `2`, `threadIdx.x` ranges from `0` to `1` (zero-based, i.e. from `0` to `blockDim.x.x - 1`).
      - We calculate the global ID for a `thread` as `int idx = threadIdx.x + blockIdx.x * blockDim.x`.
        - We have `gridDim.x * blockDim.x = 3 * 2 = 6` threads in total, so `idx` ranges from `0` to `5` (zero-based).
      - We print out the values of `gridDim.x`, `blockDim.x`, `blockIdx.x`, `threadIdx.x`, and `idx`.
        - This shows us which `thread` is running this specific copy of the kernel function, which `block` it is in, etc.
      - We have a `boundary guard`, i.e. `if(idx >= n) return;`
        - The ensures we don't index into an array with `idx` if `idx` is out of bounds.
        - We have `6` threads in total, and `5` (defined by `N`) elements in each array, so `idx = 5` is out of bounds.
      - Finally, we copy one element from the `input` array into the `output` using `idx` as the index.
        - Remember, each thread runs its own `copy` of the kernel function (in parallel), each with the same set of kernel function `arguments`.

    ```c
    __global__ void kernel(int *input, int *output, int n)
    {       
        int idx = threadIdx.x + blockIdx.x * blockDim.x;
        
        printf("gridDim.x = %d, blockDim.x = %d, blockIdx.x = %d, threadIdx.x = %d, idx = %d\n", gridDim.x, blockDim.x, blockIdx.x, threadIdx.x, idx);
        
        if(idx >= n)
        {
            printf("Boundary checking avoided indexing outside of the arrays [idx = %d]\n", idx);
            return;
        }

        output[idx] = input[idx];
    }
    ```
- In the `main()` function:
  - We seed the pseudorandom number generator with the value `0` so the random numers we create will be the same every time we run the program.

    ```c
    srand(0);
    ```

  - We declare:
    - `int` pointer variables `h_input` and `h_output` for the two arrays, which will point to heap memory (RAM) on the host (CPU).
    - `int` pointer variables `d_input` and `d_output` for the two arrays, which will point to global memory on the device (GPU).
    - `int` variable `data_size` and initialize it to `N * sizeof(int)`, i.e. the total size of each array in bytes (with `N` elements of type `int` in each).
    
    ```c
    int *h_input, *h_output;
    int *d_input, *d_output;
    int data_size = N * sizeof(int);
    ```
  - We allocate memory on the host (GPU) with `malloc`, storing the pointers to the memory in variables `h_input` and `h_output`.

    ```c
    h_input = (int *)malloc(data_size);
    h_output = (int *)malloc(data_size);
    ```
  - We allocate memory on the device (GPU) with `cudaMalloc` storing the pointers to the memory in variables `d_input` and `d_output`.

    ```c
    cudaMalloc((void **)&d_input, data_size);
    cudaMalloc((void **)&d_output, data_size);
    ```
  - We initialize the `h_input` array on the host (CPU) with random values using the `rand()` function.

    ```c
    for(int i = 0; i<N; i++)
    {
        h_input[i] = rand() % 100; // random integers between 0 and 99
    }
    ```
  - We copy the elements of both arrays stored in host (GPU) memory (RAM) to device (GPU) global memory with `cudaMemcpy`.
    - Its first argument is a pointer to the memory to `copy to`.
    - Its second argument is a pointer to the memory to `copy from`.
    - Its third argument is the `size (in bytes)` of memory to copy (all `N` elements in our case).
    - Its fourth argument is a symbolic constant which determines the direction of the copy operation.
      - `cudaMemcpyHostToDevice` copies memory from the host (CPU) to the device (GPU).
      - `cudaMemcpyDeviceToHost` copies memory from the device (GPU) to the host (CPU).
    - Here we are copying the `input` and `output` arrays from the host (`h_input`, `h_output`) to the device (`d_input`, `d_output`).

    ```c
    cudaMemcpy(d_input, h_input, data_size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_output, h_output, data_size, cudaMemcpyHostToDevice);
    ```
  - We are now using CUDA's `dim3` struct to define the `gridDim` (number of blocks) and `blockDim` (number of threads in each block).
    - Before we just used `int` litterals in the launch configuration `<<<1, 1>>>`, but we can also use `dim3` variables `<<<gridDim, blockDim>>>`.
    - The launch configuration supports launching `1D`, `2D`, and `3D` blocks and threads, depending on the problem we want to solve, e.g:
      - For a `1D` problem such as copying elements between two 1D arrays, we only need to use 1 dimension, which is the `x` member variable in `dim3` structs.
      - For a `2D` problem such as filtering a 2D image, we might need to use 2 dimensions, which are the `x` and `y` member variables in `dim3` structs.
      - For a `3D` problem such as filtering a 3D MRI-scan volume, we might need to use 3 dimensions, which are the `x`, `y` and `z` member variables in `dim3` structs.
      - For our 1D problem, we are only using the `x` member variable, which means the other dimensions `y` and `z` will be set to the value `1`.
    - We create a `dim3` variable `blockDim` and initialize its `x` member variable to `THREADS_PER_BLOCK_X` (member variables `y` and `z` will be set to `1`).
      - This means we have `2` threads per block since `THREADS_PER_BLOCK_X` is defined with the value `2`.
    - We create a `dim3` variable `gridDim` and initialize its `x` member variable to `(N + THREADS_PER_BLOCK_X - 1) / THREADS_PER_BLOCK_X`.
      - This means we have `3` blocks per grid since `(N + THREADS_PER_BLOCK_X - 1) / THREADS_PER_BLOCK_X = (5 + 2 -1) / 2 = 3` (and there is only ever 1 grid).
      - This (standard) construct is commonly used to ensure enough `threads` are launched to solve a problem (but can launch more `threads` than data elements).

    ```c
    dim3 blockDim(THREADS_PER_BLOCK_X);
    dim3 gridDim((N + THREADS_PER_BLOCK_X - 1) / THREADS_PER_BLOCK_X);
    ```
  - Next, we launch the kernel with:
    - Launch configuration `<<<gridDim, blockDim>>>`, where `gridDim` and `blockDim` are our two `dim3` variables.
    - Argument list `d_input, d_output, N`  `d_input`, where `d_input` and `d_output` are the `int` pointers to the arrays on the device (GPU), and `N` the number of elements.

    ```c
    kernel<<<gridDim, blockDim>>>(d_input, d_output, N);
    ```
  - We copy the elements in the `d_output` array on the device (GPU) back to the array `h_output` on the host (CPU) using `cudaMemcpy`.
    - Notice the final argument is now `cudaMemcpyDeviceToHost`, i.e. the direction of the copy operation is from the device (GPU) to the host (CPU).

    ```c
    cudaMemcpy(h_output, d_output, data_size, cudaMemcpyDeviceToHost);   
    ```
  - Then we print out the elements in the two arrays `h_input` and `h_output` on the host (CPU).
   
    ```c
    printf("\n%-5s   %-6s\n", "input", "output");
    for(int i = 0; i<N; i++)
    {
        printf("%-5d   %-6d\n", h_input[i], h_output[i]);
    }
    ```
  - Finally, we free the mmeory allocated for the arrays:
    - We free the `int` pointers (`d_input` and `d_output`) pointing to memory on the device (GPU) with `cudaFree`.
    - We free the `int` pointers (`h_input` and `h_output`) pointing to memory on the host (CPU) with `free`.
    - Both functions take a pointer to the memory 
    - Notice the naming convention used in this program for pointers to memory on the host (`h_` prefix) and the device (`d_` prefix).

    ```c
    cudaFree(d_input);
    cudaFree(d_output);
    free(h_input);
    free(h_output);
    ```
- Run the cell below to see the output from the program.

**TL;DR**
- A kernel launch `kernel<<<blocks, threads>>>(argumentlist)` has:
  - A launch configuration `<<<gridDim, blockDim>>>` that specifies how man `blocks` (`gridDim`) and `threads` per `block` (`blockDim`) to launch.
    - It accepts `int` parameters, e.g. `<<<3, 2>>>` or `dim3` parameters, e.g. `<<<gridDim, blockDim>>>`.
    - `dim3` is a struct containing `int` member variables `x`, `y`, and `z`, used to structure `blocks` and `threads` for `1D`, `2D`, or `3D` problems.
  - An argument list `argumentlist` which must match the kernel function's parameter list.
    - Each `thread` runs a `copy` of the same kernel function, with the exact same `argumentlist`, in parallel (at the same time).
- A kernel function is run for each `thread`, where each `thread` has access to 4 global `dim3` variables `gridDim`, `blockDim`, `blockIdx`, and `threadIdx`
  - `gridDim` and `blockDim` are from the launch configuration and contain the number of `blocks` (`gridDim`) and number of `threads` per `block` (`blockDim`).
  - `blockIdx` and `threadIdx` contain unique `block` IDs within a grid (`blockIdx`) and unique `thread` IDs within a `block` (`threadIdx`).
- The construct `(N + THREADS_PER_BLOCK_X - 1) / THREADS_PER_BLOCK_X`:
  - Is commonly used ensure enough (at least as many) `threads` are launched needed to solve a problem (cover all data elements).
  - But can launch more `threads` than the total number of data elements.
- Since we can have more `threads` than data elements, we **always use bounday guards in CUDA kernels** to avoid out-of-bounds indexing.
- We use `malloc` and `free` for managing memory on the host (CPU).
- We use `cudaMalloc` and `cudaFree` for managing memory on the device (GPU).
- We use `cudaMemcpy` to copy memory between the host (CPU) and device (GPU), where the fourth argument determines the direction of the copy operation.
  - `cudaMemcpyHostToDevice` copies memory from the `host` (CPU) to the `device` (GPU).
  - `cudaMemcpyDeviceToHost` copies memory from the `device` (GPU) to the `host` (CPU).

In [26]:
%%cuda
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <cuda_runtime.h>

#define N 5
#define THREADS_PER_BLOCK_X 2

__global__ void kernel(int *input, int *output, int n)
{       
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    
    printf("gridDim.x = %d, blockDim.x = %d, blockIdx.x = %d, threadIdx.x = %d, idx = %d\n", gridDim.x, blockDim.x, blockIdx.x, threadIdx.x, idx);
    
    if(idx >= n)
    {
        printf("Boundary checking avoided indexing outside of the arrays [idx = %d]\n", idx);
        return;
    }

    output[idx] = input[idx];
}

int main(void)
{
    srand(0);
    
    int *h_input, *h_output;
    int *d_input, *d_output;
    int data_size = N * sizeof(int);

    h_input = (int *)malloc(data_size);
    h_output = (int *)malloc(data_size);

    cudaMalloc((void **)&d_input, data_size);
    cudaMalloc((void **)&d_output, data_size);

    for(int i = 0; i<N; i++)
    {
        h_input[i] = rand() % 100;
    }

    cudaMemcpy(d_input, h_input, data_size, cudaMemcpyHostToDevice);   
    cudaMemcpy(d_output, h_output, data_size, cudaMemcpyHostToDevice);   

    dim3 blockDim(THREADS_PER_BLOCK_X);
    dim3 gridDim((N + THREADS_PER_BLOCK_X - 1) / THREADS_PER_BLOCK_X);

    // Launch kernel() function
    kernel<<<gridDim, blockDim>>>(d_input, d_output, N);

    cudaMemcpy(h_output, d_output, data_size, cudaMemcpyDeviceToHost);   

    printf("\n%-5s   %-6s\n", "input", "output");
    for(int i = 0; i<N; i++)
    {
        printf("%-5d   %-6d\n", h_input[i], h_output[i]);
    }

    cudaFree(d_input);
    cudaFree(d_output);
    free(h_input);
    free(h_output);

    return 0;
}

gridDim.x = 3, blockDim.x = 2, blockIdx.x = 2, threadIdx.x = 0, idx = 4
gridDim.x = 3, blockDim.x = 2, blockIdx.x = 2, threadIdx.x = 1, idx = 5
gridDim.x = 3, blockDim.x = 2, blockIdx.x = 1, threadIdx.x = 0, idx = 2
gridDim.x = 3, blockDim.x = 2, blockIdx.x = 1, threadIdx.x = 1, idx = 3
gridDim.x = 3, blockDim.x = 2, blockIdx.x = 0, threadIdx.x = 0, idx = 0
gridDim.x = 3, blockDim.x = 2, blockIdx.x = 0, threadIdx.x = 1, idx = 1
Boundary checking avoided indexing outside of the arrays [idx = 5]

input   output
83      83    
86      86    
77      77    
15      15    
93      93    



### Inspecting the Output

- In the output we see:
  - `gridDim.x` is `3`, i.e. there a `3` `blocks` in the grid.
  - `blockDim.x` is `2`, i.e. there a `2` `threads` in each `block`.
  - `blockIdx.x` varies from `0` to `2`, i.e. from `0` to `gridDim.x - 1`, and is a `block`'s unique ID (i.e. unique within a kernel launch).
  - `threadIdx.x` varies from `0` to `1`, i.e. from `0` to `blockDim.x - 1`, and is a `thread`'s unique block ID (i.e. unique within a block).
  - `idx` varies from `0` to `5`, i.e. from `0` to `gridDim.x * blockDim.x`, and is a `thread`'s unique global ID (i.e. unique within a kernel launch).
  - The boundary guard was triggered for one thread, i.e. the thread with `idx = 5`, because we only have `N = 5` elements in each array.
    - So, we can have more threads running than elements in our data/arrays, why **we should always make use of boundary guards in our CUDA kernels**.
  - The `input` and `output` arrays have the same element values, so our CUDA kernel's logic is functionally correct.

---
## 2.5 Unified Memory

- CUDA supports using `Unified Memory` on most modern NVidia GPUs.
  - `Unified Memory` is mapped to the same address space on the CPU and GPU, which means we don't need to explicitly copy memory between the CPU and GPU with `cudaMemcpy`.
  - If this example doesn't work, your GPU doesn't support `Unified Memory`.
- The `#include`s and `#define`s are the same as in the previous example.
  - `cuda_runtime` also includes the function prototype for `cudaMallocManaged`, which we use for allocating `Unified Memory`.
- The kernel function is the same as before, but without the code for printing blocks, threads, etc.
  - We are just copying the elements between the two arrays now.

    ```c
    __global__ void kernel(int *input, int *output, int n)
    {       
        int idx = threadIdx.x + blockIdx.x * blockDim.x;
        output[idx] = input[idx];
    }
    ```

- The only modifications in the `main()` function are:
  - We declare two `int *` pointer variables `input` and `output`.
    - Notice we don't have `h_input`, `h_output`, `d_input`, and `d_output`, since `unified memory` is shared by the host (CPU) and the device (GPU).
    - We also declare and initialize the `data_size` in bytes of the elements in the arrays, just as before (same code).
      
    ```c
    int *input, *output;
    int data_size = N * sizeof(int);
    ```
  - Then memory is allocated in `unified memory` using the function `cudaMallocManaged()`, that takes a `void **` pointer and an `int` size (in bytes) as parameters.
    - The address of each `int` pointer variable (`&input` and `&output`) is typecast using `(void **)` for the first argument to the function.
    - The `data_size` (in bytes) is passed as the second argument to the function.
    - The function assigns an address to the memory allocated in `unified memory` to each of the `int` pointer varaibles `input` and `output`.
      - Now these pointer variables are pointing to memory in `unified memory`.
        
    ```c       
    cudaMallocManaged((void **)&input, data_size);
    cudaMallocManaged((void **)&output, data_size);
    ```
  - Next, we initialize the `input` array with random values, just as before (the only difference in the name of the pointer, which is `input` instead of `h_input`).

    ```c
    for(int i = 0; i<N; i++)
    {
        input[i] = rand() % 100;
    }
    ```
  - In the previous code, we used `cudaMemcpy` to copy memory from the host (CPU) to the device (GPU), but we don't need to do that here when using `unified memory`.
  - The code for the `dim3` launch parameter variables is the same as in the prevous code, so no need to reiterate that code here.
  - We then launch the kernel just as before, the only difference being the name of the pointers, which are `input` and `output` instead of `d_input` and `d_output`.
  
    ```c
    kernel<<<gridDim, blockDim>>>(input, output, N);
    ```
  - Now, since we aren't using `cudaMemcpy()`, we need to call `cudaDeviceSynchronize()` to block the `main()` thread on the host until the kernel function on the device completes.
  
    ```c
    cudaDeviceSynchronize();
    ```
  - Then we print out the values of the `input` and `output` arrays as before.
    - The only difference is the name of the variables, which are `input` and `output` instead of `h_input` and `h_output`.
  
    ```c
    printf("\n%-5s   %-6s\n", "input", "output");
    for(int i = 0; i<N; i++)
    {
        printf("%-5d   %-6d\n", input[i], output[i]);
    }
    ```
  - We free the allocated `unified memory` using the function `cudaFree()`, just as before.
    - The only difference is the name of the variables, which are `input` and `output` instead of `h_input`, `h_output`, `d_input` and `d_output`.
      
    ```c
    cudaFree(input);
    cudaFree(output);
    ```
- Run the cell below to see the output (it will be the same as before, except for the print-outs of the blocks, threads, etc.).

**TL:DR**

- Memory is allocated in `unified memory` using the function `cudaMallocManaged()` and is analogous to the normal C function `malloc()`.
- The function `cudaDeviceSynchronize()` is used to block the host's main thread until the kernel completes.
- Memory is deallocated (freed) in `unified memory` using the function `cudaFree()` and is analogous to the normal C function `free()`.

In [27]:
%%cuda
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <cuda_runtime.h>

#define N 5
#define THREADS_PER_BLOCK_X 2

__global__ void kernel(int *input, int *output, int n)
{       
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    output[idx] = input[idx];
}

int main(void)
{
    srand(0);
    
    int *input, *output;
    int data_size = N * sizeof(int);

    cudaMallocManaged((void **)&input, data_size);
    cudaMallocManaged((void **)&output, data_size);

    for(int i = 0; i<N; i++)
    {
        input[i] = rand() % 100;
    }

    dim3 blockDim(THREADS_PER_BLOCK_X);
    dim3 gridDim((N + THREADS_PER_BLOCK_X - 1) / THREADS_PER_BLOCK_X);

    // Launch kernel() function
    kernel<<<gridDim, blockDim>>>(input, output, N);

    cudaDeviceSynchronize();

    printf("\n%-5s   %-6s\n", "input", "output");
    for(int i = 0; i<N; i++)
    {
        printf("%-5d   %-6d\n", input[i], output[i]);
    }
    printf("\n");

    cudaFree(input);
    cudaFree(output);

    return 0;
}


input   output
83      83    
86      86    
77      77    
15      15    
93      93    




### Inspecting the Output

- In the output we see that the result is the same as before (the only difference is the abscense of the print-outs of blocks, threads, etc..

---
## 2.6 Error Checking

- CUDA supports checking for errors in device (GPU) code and from calling any CUDA function.
- The `#include`s, `#define`s, and the kernel function are the same as in the previous example.
  - `stdlib.h` also includes the function prototype for `exit` and the symbolic constant `EXIT_FAILURE` used in this example.
  - `cuda_runtime` also includes:
    - The the function prototypes for `cudaMalloc`, `cudaFree`, and `cudaMemcpy`, since we are back to NOT using `unified memory`.
    - The function prototypes for `cudaGetLastError` and `cudaGetErrorString`, which we use for checking errors from CUDA function calls.
    - The typedef `cudaError_t` and symbolic constant `cudaSuccess`, also used for checking CUDA errors.
- The only modifications in the `main()` function are:
  - We are using the code from the example we used before the `unified memory` example, where we don't use `unified memory`.
  - We have wrapped all CUDA function calls as arguments in a function called `checkCuda()`, explained below, e.g.

    ```c
    checkCuda(cudaMalloc((void **)&d_input, data_size), "cudaMalloc");
    ```
  - After the kernel launch, we use the code below to check for CUDA errors in the kernel function.

    ```c
    checkCuda(cudaGetLastError(), "kernel");
    ```
  - Then, after freeing all allocated memory, we deliberately produce an error:
    - We call `cudaFree()` on the `d_output` pointer twice, which produces an error the second time since that memory has already been freed.
      - All CUDA functions return a value of type `cudaError_t` which can be checked to see if an error occured in CUDA code on the device (GPU).
      - The only CUDA operation that doesn't have a return value of type `cudaError_t` is the kernel launch.
      - For that reason, CUDA provides the function `cudaGetLastError()` which will return the latest `cudaError_t` (we can use it after any CUDA function call). 
    ```c
    checkCuda(cudaFree(d_output), "cudaFree");
    ```
- We have wrapped all CUDA function calls, returning a value of type `cudaError_t`, in the function `checkCuda()`.
  - We pass the `cudaError_t` value as the first argument, and a string message as the second (the wrapped function name has been used).
- The function `checkCuda()` is defined by ourselves (it's been placed between the kernel function and the `main()` function the sample code):
  - It takes a CUDA error (`cudaError_t`) as its first argument and a message (string) as its second argument, returning `void`.
  - It checks if the value of the `cudaError_t` error is different from the symbolic constant `cudaSuccess` (`cudaSuccess` means there is no error).
    - If so, it retrieves a string-representation of the error by calling the function `cudaGetErrorString()`, which takes the error as an argument.
    - Then the error string is printed out together with an optional message (second argument to `checkCuda()`).
    - Finally, it terminates the program by calling the `exit()` function, passing in the symbolic constant `EXIT_FAILURE` as the return value to the operating system.
  - We can use `checkCuda()` by wrapping it around any CUDA function call, e.g. `checkCuda(cudaFree(d_output), "cudaFree")`.
    - Error checking will not be used going forward in this notebook to make the examples clearer, but good practice is to check for errors after each CUDA function call.

    ```c
    void checkCuda(cudaError_t err, const char *msg)
    {
        if (err != cudaSuccess)
        {
            printf("Error: %s (%s)\n", msg, cudaGetErrorString(err));
            exit(EXIT_FAILURE);
        }
    }
    ```
- Run the cell below to see the output (it will be the same as before, except for the error message that we deliberately produced).

**TL:DR**

- Each CUDA function returns a value of type `cudaError_t`.
  - If it's value is different from the symbolic constant `cudaSuccess`, and error occurred.
  - We can retrieve a string representation of a CUDA error by passing a `cudaError_t` instance as an argument the the function `cudaGetErrorString`.
  - We can retrieve the last CUDA error after a CUDA function call, including the kernel launch, using the function `cudaGetLastError`.
- We can exit from a C program prematurely, by calling the C function `exit`, passing in an exit code to the operating system.
  - The symbolic constant `EXIT_FAILURE` can be used as an exit code, representing a general error.

In [28]:
%%cuda
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <cuda_runtime.h>

#define N 5
#define THREADS_PER_BLOCK_X 2

__global__ void kernel(int *input, int *output, int n)
{       
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    output[idx] = input[idx];
}

// Error checking
void checkCuda(cudaError_t err, const char *msg)
{
    if (err != cudaSuccess)
    {
        printf("Error: %s (%s)\n", msg, cudaGetErrorString(err));
        exit(EXIT_FAILURE);
    }
}

int main(void)
{
    srand(0);
    
    int *h_input, *h_output;
    int *d_input, *d_output;
    int data_size = N * sizeof(int);

    h_input = (int *)malloc(data_size);
    h_output = (int *)malloc(data_size);

    checkCuda(cudaMalloc((void **)&d_input, data_size), "cudaMalloc");
    checkCuda(cudaMalloc((void **)&d_output, data_size), "cudaMalloc");

    for(int i = 0; i<N; i++)
    {
        h_input[i] = rand() % 100;
    }

    checkCuda(cudaMemcpy(d_input, h_input, data_size, cudaMemcpyHostToDevice), "cudaMemcpy");
    checkCuda(cudaMemcpy(d_output, h_output, data_size, cudaMemcpyHostToDevice), "cudaMemcpy");

    dim3 blockDim(THREADS_PER_BLOCK_X);
    dim3 gridDim((N + THREADS_PER_BLOCK_X - 1) / THREADS_PER_BLOCK_X);

    // Launch kernel() function
    kernel<<<gridDim, blockDim>>>(d_input, d_output, N);

    checkCuda(cudaGetLastError(), "kernel");

    checkCuda(cudaMemcpy(h_output, d_output, data_size, cudaMemcpyDeviceToHost), "cudaMemcpy");

    printf("\n%-5s   %-6s\n", "input", "output");
    for(int i = 0; i<N; i++)
    {
        printf("%-5d   %-6d\n", h_input[i], h_output[i]);
    }
    printf("\n");

    checkCuda(cudaFree(d_input), "cudaFree");
    checkCuda(cudaFree(d_output), "cudaFree");
    free(h_input);
    free(h_output);

    checkCuda(cudaFree(d_output), "cudaFree");

    return 0;
}


input   output
83      83    
86      86    
77      77    
15      15    
93      93    

Error: cudaFree (invalid argument)



### Inspecting the Output

- In the output we see that the result is the same as before (the only difference is that we are NOT using `unified memory`).
- We also see the error message `invalid argument` returned from `cudaFree()` when we try to deallocate device (GPU) memory that has already been freed.

---
## 2.7 Measuring Execution Time on the Host (CPU) and on the Device (GPU)

- A common workflow is to first implement an algorithm in a function on the host (CPU), and then in a kernel on the device (GPU).
  - The CPU version can act as a baseline benchmark for GPU kernel performance.
  - The CPU version can be used to verify the results of a GPU kernel.
  - For inexperienced manycore programmers, it's often easier to start with a CPU version, and then convert it into a GPU version.
- Let's use the same code as before, but instrument it with timing code, wrapped around the CPU function call and around the GPU kernel launch.
- The imported header files are the same as before:
  - `stdlib.h` also contains the function prototype for `clock`, the `clock_t` typedef, and the symbolic constant `CLOCKS_PER_SEC`.
    - `clock()` is a parameterless function returning a value of type `clock_t`.
    - `clock_t` contains the number of `ticks` elapsed since the program started.
    - `CLOCKS_PER_SEC` is defined as the number of `ticks` in a second (`ticks / CLOCKS_PER_SEC * 1000.0` converts `ticks` to milliseconds).
  - `cuda_runtime.h` contains a typedef `cudaEvent_t` and prototypes `cudaEventCreate`, `cudaEventRecord`, `cudaEventElapsedTime`, `cudaEventSynchronize`, and `cudaEventDestroy`.
    - `cudaEvent_t` represent a CUDA event, e.g. `cudaEvent_t start` (we won't explore CUDA events (or CUDA streams) in detail in this notebook).
    - `cudaEventCreate` is used to initialize a CUDA event, e.g. `cudaEventCreate(&start)`
    - `cudaEventRecord` is used to start recording (monitoring) a CUDA event, e.g. `cudaEventRecord(start)`
    - `cudaEventElapsedTime` is used to compute and return the elapsed time in milliseconds between to CUDA events, e.g. `cudaEventElapsedTime(&elapsed_ms, start, stop)`
    - `cudaEventSynchronize` blocks the CPU's main thread until an event has completed (in our code, when the kernel is done), e.g. `cudaEventSynchronize(stop)`
    - `cudaEventDestroy` frees (destroys) a CUDA event, e.g. `cudaEventDestroy(start)`
    
- We define a host (CPU) function `copy()`, equivalent to the device (GPU) kernel function `kernel()`
  - The GPU kernel function is the same as before.
    
    ```c
    void copy(int *input, int *output, int n)
    {
        for(int idx = 0; idx < n; idx++)
        {
            output[idx] = input[idx];
        }
    }
    ```
- In the `main()` function:
  - We wrap the code below around the device (GPU) kernel launch `kernel()`.
    - First we declare two CUDA event variables `start` and `stop`, and initialize them with `cudaEventCreate(&start)` and `cudaEventCreate(&stop)`.
    - Then we record the `start` event with `cudaEventRecord(start)` (this records the current time in the kernel and stores it in the `start` event).
    - Next, the device (GPU) kernel is launched as usual.
    - Then we record the `stop` event with `cudaEventRecord(stop)` (this records the current time in the kernel and stores it in the `stop` event).
    - We call `cudaEventSynchronize(stop)` to block the host (CPU) main thread until the `stop`event is done (i.e. until the kernel is done).
    - Finally, we declare a variable `float gpu_elapsed_ms` and pass it to the function `cudaEventElapsedTime(&gpu_elapsed_ms, start, stop);`
      - We also pass in `start`and `stop`, where the function will store the elapsed time in milliseonds in the variable `gpu_elapsed_ms`.

    ```c
    // --------------------------------------------------------------
    // Timing the device (GPU) kernel execution time
    // --------------------------------------------------------------
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start);
  
    // Device kernel() launch
    kernel<<<gridDim, blockDim>>>(d_input, d_output, N);
  
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    float gpu_elapsed_ms;
    cudaEventElapsedTime(&gpu_elapsed_ms, start, stop);
    // --------------------------------------------------------------
    ```
  - We wrap the code below around the host (CPU) function call `copy()`.
    - We call the function `clock()` to record the current number of `ticks` since the program started, and store the result in a variable `cpu_start` of type `clock_t`.
    - Then we call the host (CPU) function `copy()`.
    - Next, we call the function `clock()` again to record the current number of `ticks` again and store the result in a variable `cpu_stop` of type `clock_t`.
    - Finally, we calculate the elapsed number of milliseconds as `float cpu_elapsed_ms = (double)(cpu_stop - cpu_start) / CLOCKS_PER_SEC * 1000.0`.
  
    ```c
    // --------------------------------------------------------------
    // Timing the host (CPU) function execution time
    // --------------------------------------------------------------
    clock_t cpu_start = clock();

    // Host function call
    copy(h_input, h_output_cpu, N);
    
    clock_t cpu_stop = clock();
    float cpu_elapsed_ms = (double)(cpu_stop - cpu_start) / CLOCKS_PER_SEC * 1000.0;
    // --------------------------------------------------------------
    ```
  - We print out the execution time for the device (GPU) kernel and host (GPU) function.

    ```c
    printf("GPU execution time  : %f ms\n", gpu_elapsed_ms);
    printf("CPU execution time  : %f ms\n", cpu_elapsed_ms);
    ```
  - We print out the execution time for the device (GPU) kernel and host (GPU) function.

    ```c
    printf("GPU execution time  : %f ms\n", gpu_elapsed_ms);
    printf("CPU execution time  : %f ms\n", cpu_elapsed_ms);
    ```
  - We use a separate `int` pointer variable `h_output_cpu` for storing the output from the host (CPU) function call.
    - We verify the output results from the device (GPU) kernel and host (CPU) function are the same.
    - This is a common best practice when verifying the correct functionality of an algorithm implemented in a device (GPU) kernel.
      - We use the `abs()` function to compute the absolute difference between each eleement pair in the two arrays.
      - If we were using `float`s instead of `int`s, we can use the `fabs()` function and compare the difference to e.g. `1e-5`.

    ```c
    int errorsum = 0;
    for (int i = 0; i < N; i++)
    {
        int error = abs(h_output[i] - h_output_cpu[i]);
        if (error > 0)
        {
            //printf("Result verification failed for element with index %d!\n", i);
            errorsum += error;
        }
    }
    // Print verification result
    printf("\nVerification : %s\n", (errorsum > 0) ? "FAILED" : "PASSED");
    ```
  - We also print out the two arrays as before (same code) after launching the device (GPU) kernel and after calling the host (CPU) function.
  - Lastly, we also free all memory, including the two CUDA events.

    ```c
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    ```
- Run the cell below to see the output.
  - We won't record time in this notebook going forward to make the example code clearer, but now you know how to do it yourself-

**TL:DR**

- We can measure the execution time for a CUDA kernel with types and function prototypes declared in `cuda_runtime.h`.
- We can measure the execution time for a C function (or any C code) with types and function prototypes declared in `stdlib.h`.
- We can cmopute the absolute difference between two results to determine if they are correct (given at least one of the results is correct).

In [29]:
%%cuda
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <cuda_runtime.h>

#define N 5
#define THREADS_PER_BLOCK_X 2

// Device kernel
__global__ void kernel(int *input, int *output, int n)
{       
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    output[idx] = input[idx];
}

// Host function
void copy(int *input, int *output, int n)
{
    for(int idx = 0; idx < n; idx++)
    {
        output[idx] = input[idx];
    }
}

int main(void)
{
    srand(0);
    
    int *h_input, *h_output, *h_output_cpu;
    int *d_input, *d_output;
    int data_size = N * sizeof(int);

    h_input = (int *)malloc(data_size);
    h_output = (int *)malloc(data_size);
    h_output_cpu = (int *)malloc(data_size);

    cudaMalloc((void **)&d_input, data_size);
    cudaMalloc((void **)&d_output, data_size);

    for(int i = 0; i<N; i++)
    {
        h_input[i] = rand() % 100;
    }

    cudaMemcpy(d_input, h_input, data_size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_output, h_output, data_size, cudaMemcpyHostToDevice);

    dim3 blockDim(THREADS_PER_BLOCK_X);
    dim3 gridDim((N + THREADS_PER_BLOCK_X - 1) / THREADS_PER_BLOCK_X);

    // --------------------------------------------------------------
    // Timing the device (GPU) kernel execution time
    // --------------------------------------------------------------
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start);
  
    // Device kernel() launch
    kernel<<<gridDim, blockDim>>>(d_input, d_output, N);
  
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    float gpu_elapsed_ms;
    cudaEventElapsedTime(&gpu_elapsed_ms, start, stop);
    // --------------------------------------------------------------

    cudaMemcpy(h_output, d_output, data_size, cudaMemcpyDeviceToHost);

    // Print measured device kernel execution time
    printf("GPU execution time  : %f ms\n", gpu_elapsed_ms);

    // Print elements in both arrays
    printf("\n%-5s   %-6s\n", "input", "output");
    for(int i = 0; i<N; i++)
    {
        printf("%-5d   %-6d\n", h_input[i], h_output[i]);
    }
    printf("\n");

    cudaFree(d_input);
    cudaFree(d_output);

    // --------------------------------------------------------------
    // Timing the host (CPU) function execution time
    // --------------------------------------------------------------
    clock_t cpu_start = clock();

    // Host function call
    copy(h_input, h_output_cpu, N);
    
    clock_t cpu_stop = clock();
    float cpu_elapsed_ms = (double)(cpu_stop - cpu_start) / CLOCKS_PER_SEC * 1000.0;
    // --------------------------------------------------------------

    // Print measured host function execution time
    printf("CPU execution time  : %f ms\n", cpu_elapsed_ms);

    // Print elements in both arrays
    printf("\n%-5s   %-6s\n", "input", "output");
    for(int i = 0; i<N; i++)
    {
        printf("%-5d   %-6d\n", h_input[i], h_output_cpu[i]);
    }

    // Verify the results in the GPU output with the CPU output
    int errorsum = 0;
    for (int i = 0; i < N; i++)
    {
        int error = abs(h_output[i] - h_output_cpu[i]);
        if (error > 0)
        {
            //printf("Result verification failed for element with index %d!\n", i);
            errorsum += error;
        }
    }
    
    // Print verification result
    printf("\nVerification : %s\n", (errorsum > 0) ? "FAILED" : "PASSED");

    free(h_input);
    free(h_output);
    free(h_output_cpu);

    cudaEventDestroy(start);
    cudaEventDestroy(stop);

    return 0;
}

GPU execution time  : 0.082944 ms

input   output
83      83    
86      86    
77      77    
15      15    
93      93    

CPU execution time  : 0.001000 ms

input   output
83      83    
86      86    
77      77    
15      15    
93      93    

Verification : PASSED



### Inspecting the Output

- In the output we see that the execution time on the GPU is slower than on the CPU.
- This is expected since copying 5 elements from one array to another is just a waste of time on a GPU.
  - **Not all problems are suitable for a GPU, in which case we should use the CPU instead**.
- We also see the results verification `PASSED` so we can rest assured that the kernel function is correct (if the CPU function correct, of course).

---
## 2.8 Shared Memory and Thread Synchronization on the Device (GPU)

- `Shared memory` is a fast, low-latency memory located on-chip, accessible by all `threads` in a `block`.
  - **Location**: On-chip, accessible by all `threads` in a `block`.
  - **Access**: Readable and writeable by all `threads` in a `block` (also `writable` from the `host` before kernel launch).
  - **Size limit**: Typically `48 KB` per `SM` (Streaming Multiprocessor).
  - **Speed**: Very fast, much faster than `global memory`.
  - **Scope**: Shared only among `threads` in the same `block`.
  - **Lifetime**: Exists for the duration of the `block`.
- Use `shared memory` when:
  - Threads need to cooperate, such as tiling, caching, or communication between `threads`.
    - `Shared memory` is specified within a kernel function with the qualifier `__shared__`.
    - It can be initialized `statically` or `dynamically`.
- `Thread synchronization` is used to synchronize `threads`, especially `threads`in a `block` when using `shared memory`:
  - Purpose: Barrier synchronization — all `threads` in the `block` must reach it before any can proceed.
    - A barrier (thread synchronization) is done with the command `__syncthreads()` in kernel function.
  - Ensures all `shared memory` reads/writes are complete before continuing.
  - Used to prevent `race conditions`.
- Issues to be aware of when using `shared memory`:
  - `Non-coalesced memory access` also applies when accessing `global memory` (actually more important in that case).
  - `Warp (thread) divergence` applies to `threads` within the same `warp` (a `warp` is a group of `32` threads that are scheduled to run on `SPs` within a `SM`/`block`).
  - `Low occupancy` is related to `shared memory` but applies more generally to a `kernel launch`.

  | Issue                       | Consequence                        | Fix                                |
  | --------------------------- | ---------------------------------- | ---------------------------------- |
  | Race conditions             | Wrong results                      | Use `__syncthreads()` or atomics   |
  | No synchronization          | Inconsistent reads/writes          | Use `__syncthreads()`              |
  | [Bank conflicts](https://www.youtube.com/watch?v=CZgM3DEBplE)              | Performance slowdown               | Pad arrays, restructure access     |
  | Exceeding memory limit      | Kernel launch fails or runs slower | Reduce usage, use fewer threads    |
  | Wrong indexing              | Wrong data or crash                | Use `threadIdx` properly           |
  | Uninitialized/out-of-bounds | Undefined behavior                 | Always initialize and guard bounds |
  | [Non-coalesced memory access](https://www.youtube.com/watch?v=mLxZyWOI340&list=PLAwxTw4SYaPnFKojVQrmyOGFCqHTxfdv2&index=97)| Slower execution speed | Coalesce memory access |
  | [Warp (thread) divergence](https://www.youtube.com/watch?v=bHkFV-YMxxY&list=PLAwxTw4SYaPnFKojVQrmyOGFCqHTxfdv2&index=106) | Slower execution speed | Avoid branches and loops |
  | [Low occupancy](https://www.youtube.com/watch?v=2NGQvnT_3gU) | Slower execution speed | Increase occupancy |  

<br />  

- Now, let's look at a simple example of using `shared memory`.
- The code is the same as before, but with the following modifications:
  - In the `kernel` function, we declare a buffer (array) with the `__shared__` qualifier.
  - The `shared memory` can be declared with a `static` size or with a `dynamic size`.
  - In the sample code, we are using a `dynamic size`, where
    - the size isn't provided within the square brackets `[]`
    - the keyword `extern` is used infront of the `__static__` qualifier
      - this means the size id declared elsewhere (as an additional launch configuration parameter)
  - If we wanted a `static` size, we could use the commented-out row below instead, where
    - the sizs is provided within the square brackets `[THREADS_PER_BLOCK_X]` (`THREADS_PER_BLOCK_X` in this case).

  ```c
  extern __shared__ int shared[];               // dynamic size
  //__shared__ int shared[THREADS_PER_BLOCK_X]; // static size
  ```
- Let's look at the complete kernel function:
  - At the top, we decalare `shared memory` with a dynamic size.
  - Then we calculate a `thread`'s global index/ID (`g_idx`) and a `thread`'s local/shared index/ID (`s_idx`).
    - We have to be careful in how we use the threads for indexing (`g_idx` is unique within a kernel launch, `s_id` is unique within a `block` on the same `SM`).
    - Remember, if we have `blockDim.x` `threads` per `block` (with a `s_id` ranging from `0` to `blockDim.x` - 1).
  - Our usual `boundary guard` comes next `if(g_idx >0 n)`.
  - Then we copy elements from the `input` array into `shared` memory.
    - The index into the `input` array is `g_idx`.
    - The index into the `shared` array is `s_idx`.
    - Different indexing schemes might ne necessary depending on the problem/algorithm.
  - Next, we have a thread barrier `__syncthreads()`.
    - This ensures no `thread` within the `block` can continue past this row until all `threads` in the `block` have completed the code above this row.
      - This is important, since some `threads`might not have copied their element from the `input` array into the `shared` array yet.
      - In this example, it isn't an issue, because no other `thread` will read another `thread`'s element in the `shared` array in the code below the barrier `__syncthreads`.
      - For other problems, this might not be the case, so if `threads` aren't synchronized, they might continue and read stale data from the `shared` array.
  - Lastly, when all `threads` are synchronized, a `thread` copies an element from the `shared` array into the `output` array.
    - The index into the `output` array is `g_idx`.
    - The index into the `shared` array is `s_idx`.
    - Different indexing schemes might ne necessary depending on the problem/algorithm.

  ```C
  __global__ void kernel(int *input, int *output, int n)
  {
      // Shared memory
      extern __shared__ int shared[];               // dynamic size
      //__shared__ int shared[THREADS_PER_BLOCK_X]; // static size
      
      int g_idx = threadIdx.x + blockIdx.x * blockDim.x; // index in global memory (globally unique)
      int s_idx = threadIdx.x;                           // index in shared memory (unique within a block)

      if(g_idx >= n) return; // boundary guard

      // Copy elements in global memory (input) to shared memory (shared)
      shared[s_idx] = input[g_idx];

      // Synchronize threads
      __syncthreads();       // all threads in the same block must be done with the operations above before any thread can continue

      // Copy elements in shared memory (shared) to global memory (output)
      output[g_idx] = shared[s_idx];
  }
  ```
- Now, let's look at modifications in the `main()` function (most of the code is the same as before, but with the timing removed for clarity).
  - In fact, there is only one modification:
    - Since we are using a dynamic size for our `shared memory`, we first define the size of the memory with `int shared_size = THREADS_PER_BLOCK_X * sizeof(int)`.
    - Then we supply the size `shared_size` (in bytes) as a third parameter in the launch configuration `<<<gridDim, blockDim, shared_size>>>`.
    - If we were using a static size, we would comment these two rows, uncomment the last row, and use the same launch configuration as before (i.e. no change).
  - Best practice is to use a dynamic size, since we can determine a variable size in the code (without relying on e.g. a `#define` preprocessing directive).

    ```c
    // Device kernel() launch
    int shared_size = THREADS_PER_BLOCK_X * sizeof(int);
    kernel<<<gridDim, blockDim, shared_size>>>(d_input, d_output, N);
    //kernel<<<gridDim, blockDim>>>(d_input, d_output, N);

    ```
- Run the cell below to see the output (which is exactly the same as before).

**TL;DR**

- `Shared memory` can be declared using either:
  - A dynamic size
    - We use the keyword `extern`and the qualifier `__shared__` infront of the local variable in the kernel function.
    - We don't specify the size when declaring the variable in the kernel function, e.g. `extern __shared__ int shared[]`
    - We pass the size (in bytes) of the `shared memory` as a third parameter in the launch configuration `<<<blocks, threads, shared_size>>>`.
  - A static size
    - We use the qualifier `__shared__` infront of the local variable in the kernel function.
    - We include the size when declaring the variable in the kernel function, e.g. `__shared__ int shared[THREADS_PER_BLOCK_X]`
    - We call the kernel function without passing a third parameter to the launch configuration (or the value `0`) `<<<blocks, threads, shared_size>>>`.
- `Thread synchronization` is important when using `shared memory`.
  - We can synchronize `threads` with the statement `__syncthreads();`
    - No `thread` in a `block` can continue past that row until all `threads` have completed their tasks in the code above that row.
- Remember this regarding indexing:
  - A `thread`'s unique ID within a `block` is `threadIdx.x` (a specific `block` only runs on one `SM`, the `SM` with the `shared memory`).
  - A `thread`'s globally unique ID within a grid is calculated as `blockIdx.x * blockDim.x + threadIdx.x`.
- Multiple issues are related to `shared memory` (is one isn't aware of them).
  - We won't explore these issues (e.g. memory coalescence, warp divergence, bank conflicts, occupancy, etc.) in detail in this notebook.

In [30]:
%%cuda
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <cuda_runtime.h>

#define N 5
#define THREADS_PER_BLOCK_X 2

// Device kernel
__global__ void kernel(int *input, int *output, int n)
{
    // Shared memory
    extern __shared__ int shared[];               // dynamic size
    //__shared__ int shared[THREADS_PER_BLOCK_X]; // static size
    
    int g_idx = threadIdx.x + blockIdx.x * blockDim.x; // index in global memory (globally unique)
    int s_idx = threadIdx.x;                           // index in shared memory (unique within a block)

    if(g_idx >= n) return; // boundary guard

    // Copy elements in global memory (input) to shared memory (shared)
    shared[s_idx] = input[g_idx];

    // Synchronize threads
    __syncthreads();       // all threads in the same block must be done with the operations above before any thread can continue

    // Copy elements in shared memory (shared) to global memory (output)
    output[g_idx] = shared[s_idx];
}

int main(void)
{
    srand(0);
    
    int *h_input, *h_output;
    int *d_input, *d_output;
    int data_size = N * sizeof(int);

    h_input = (int *)malloc(data_size);
    h_output = (int *)malloc(data_size);

    cudaMalloc((void **)&d_input, data_size);
    cudaMalloc((void **)&d_output, data_size);

    for(int i = 0; i<N; i++)
    {
        h_input[i] = rand() % 100;
    }

    cudaMemcpy(d_input, h_input, data_size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_output, h_output, data_size, cudaMemcpyHostToDevice);

    dim3 blockDim(THREADS_PER_BLOCK_X);
    dim3 gridDim((N + THREADS_PER_BLOCK_X - 1) / THREADS_PER_BLOCK_X);
    
    // Device kernel() launch
    int shared_size = THREADS_PER_BLOCK_X * sizeof(int);
    kernel<<<gridDim, blockDim, shared_size>>>(d_input, d_output, N);
    //kernel<<<gridDim, blockDim>>>(d_input, d_output, N);

    cudaMemcpy(h_output, d_output, data_size, cudaMemcpyDeviceToHost);

    // Print elements in both arrays
    printf("\n%-5s   %-6s\n", "input", "output");
    for(int i = 0; i<N; i++)
    {
        printf("%-5d   %-6d\n", h_input[i], h_output[i]);
    }

    cudaFree(d_input);
    cudaFree(d_output);

    free(h_input);
    free(h_output);

    return 0;
}


input   output
83      83    
86      86    
77      77    
15      15    
93      93    



### Inspecting the Output

- The output is exactly the same as before (same algorithm, just using different type of memory).

---
## 2.9 Constant Memory on the Device (GPU)

- `Constant memory` is a special type of GPU memory optimized for cases where many `threads` read the same values.
  - **Location**: On-device, separate from `global memory`.
  - **Access**: Readable by all `threads` and is `read-only` from the `device`, but `writable` from `host`.
  - **Size limit**: `64 KB` (per `device`).
  - **Speed**: Very fast if all `threads` access the same address.
  - **Scope**: Globally accessible (like global variables).
  - **Lifetime**: Exists for the duration of the `kernel launch`.
- Use `constant memory` when:
  - All or most `threads` access the same data (e.g., coefficients, transformation matrices, filters).
  - The data is known before kernel launch and doesn't change during execution.
  - The data is small (<= `64 KB`).
- Let's look at a simple example using `constant memory`.
- It's the same code as before, but with the `shared memory` removed, and with the following modifications:
  - Above the kernel function (not inside it), we declare a constant memory buffer (array) using the `__constant__` qualifier.
  - In the kernel function:
    - We multiply an element in the `input` array with an elements in the `constant` array, both with the same index.
    - Then we assigning the product to the `output` array using the same index.
    - Note that we have declared the size of the `constant memory` to be the same as the number of elements `N`.
      - This is perfectly fine for this example where `N = 5`, but `constant memory` is extremely limited (small).
      - We wouldn't be able to use `N` as the `constant memory`'s size if, say, `N` was `1000000` (a million elements).

      ```c
      // constant memory
      __constant__ int constant[N];
      
      // Device kernel
      __global__ void kernel(int *input, int *output, int n)
      {   
          int idx = threadIdx.x + blockIdx.x * blockDim.x;
          
          if(idx >= n) return; // boundary guard
          
          // Multiply input elements with coefficients in constant memory and store the product in output
          output[idx] = input[idx] * constant[idx];
      }
      ```
  - In the main() function, the code is the same as before (but with `shared memory` removed), but with the following modifications:
    - We declare an `int` pointer variable on the host (CPU) to define the contents to be copied to the `constant memory`.

      ```c
      int *h_coefficients;
      ```
    - We create a variable with the same size (but in bytes) as the statically defined `constant memory`.

      ```c
      int constant_size = N * sizeof(int);
      ```
    - We allocate space in host (CPU) memory (RAM) the data with will be copying to the `constant memory`.

      ```c
      h_coefficients = (int *)malloc(constant_size);
      ```
    - We initialize the data we will be copying tp `constant memory`.
      - Notice, all the elements in `h_coefficients` are two (so the elements in the `output` array from the kernel function will be twice as large as in the `input` array).

      ```c
      for(int i = 0; i<N; i++)
      {
         h_coefficients[i] = 2;
      } 
      ```
    - Then we copy the host (CPU) memory to the device (GPU) `constant mmeory` using the CUDA function `cudaMemcpyToSymbol`.
      - Notice that we aren't using the `cudaMemcpy` function when copying host (CPU) memory to device (GPU) `constant memory`.
      - We pass in a pointer to the `constant` memory as the first argument.
      - We pass in a pointer to the host (CPU) memory `h_coefficients` as the second argument.
      - We pass in the size (in bytes) of the `constant memory` as the third argument.
    
      ```c
      cudaMemcpyToSymbol(constant, h_coefficients, constant_size); // copy host (CPU) memory to device (GPU) constant memory
      ```
    - At the very end of the `main()` function, we free the memory on the host (CPU), allocated to store the values copied to `constant memory`.

      ```c
      free(h_coefficients);
      ```
- Run the cell below to see the output.

In [31]:
%%cuda
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <cuda_runtime.h>

#define N 5
#define THREADS_PER_BLOCK_X 2

// constant memory
__constant__ int constant[N];

// Device kernel
__global__ void kernel(int *input, int *output, int n)
{   
    int idx = threadIdx.x + blockIdx.x * blockDim.x;

    if(idx >= n) return; // boundary guard

    // Multiply input elements with coefficients in constant memory and store the product in output
    output[idx] = input[idx] * constant[idx];
}

int main(void)
{
    srand(0);
    
    int *h_input, *h_output, *h_coefficients;
    int *d_input, *d_output;
    int data_size = N * sizeof(int);
    int constant_size = N * sizeof(int);

    h_input = (int *)malloc(data_size);
    h_output = (int *)malloc(data_size);
    h_coefficients = (int *)malloc(constant_size);

    cudaMalloc((void **)&d_input, data_size);
    cudaMalloc((void **)&d_output, data_size);

    for(int i = 0; i<N; i++)
    {
        h_input[i] = rand() % 100;
    }

    for(int i = 0; i<N; i++)
    {
        h_coefficients[i] = 2;
    }

    cudaMemcpy(d_input, h_input, data_size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_output, h_output, data_size, cudaMemcpyHostToDevice);
    cudaMemcpyToSymbol(constant, h_coefficients, constant_size);       // copy host (CPU) memory to device (GPU) constant memory

    dim3 blockDim(THREADS_PER_BLOCK_X);
    dim3 gridDim((N + THREADS_PER_BLOCK_X - 1) / THREADS_PER_BLOCK_X);
    
    // Device kernel() launch
    kernel<<<gridDim, blockDim>>>(d_input, d_output, N);

    cudaMemcpy(h_output, d_output, data_size, cudaMemcpyDeviceToHost);

    // Print elements in both arrays
    printf("\n%-5s   %-6s\n", "input", "output");
    for(int i = 0; i<N; i++)
    {
        printf("%-5d   %-6d\n", h_input[i], h_output[i]);
    }

    cudaFree(d_input);
    cudaFree(d_output);

    free(h_input);
    free(h_output);
    free(h_coefficients);

    return 0;
}


input   output
83      166   
86      172   
77      154   
15      30    
93      186   



### Inspecting the Output

- We see that the values in the `output` array are twice as large compared to the `input` array,

---
# 3. Sample Problems
---

## 3.1 1D Vector Addition on the Host (CPU)

<img src="images/vectoradd_cpu.png" width="500" style="float: right; margin-right: 50px;" />

Let's start with a simple problem.

Problem
  - We have three vectors (arrays) `A`, `B`, and `C`, all with `N` elements each.
  - We want to compute the elementwise sum of `A` and `B`, and store the sum in `C`.

Solution
1. Define number of elements `N=1048576`
2. Create a host function `void vectorAdd(float *A, float *B, float *C, int n)`
    - Loop through vectors `A` and `B` with `idx=0..N-1`
    - Compute `C[idx] = A[idx] + B[idx]`
3. Create a host function `main(void)`
    - Declare and allocate memory for vectors `h_A`, `h_B`, and `h_C`.
    - Initialize vectors `h_A` and `h_B` with `N` random floats each.
    - Call function `vectorAdd` with `h_A`, `h_B`, `h_C`, `N`, and measure the execution time for `vectorAdd`.
    - Verify result is correct.
    - Print execution time, verification result, and sample elements in vectors `h_A`, `h_B`, and `h_C`.
    - Free memory allocated for vectors `h_A`, `h_B`, and `h_C`.

In [32]:
%%cuda
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <math.h>

// Number of elements (1048576)
#define N (1 << 20) 

// Host function (elementwise addition of vectors A and B, placing the sum in vector C)
void vectorAdd(float *A, float *B, float *C, int n)
{   
    // Loop through vectors and compute sum C = A + B
    for (int idx = 0; idx < n; idx++)
    {
        C[idx] = A[idx] + B[idx];
    }
}

// Host main routine
int main(void)
{
    // Seed pseudorandom number generator
    srand(0);

    // Compute the size of the vectors (in bytes)
    size_t size = N * sizeof(float);

    // Declare and allocate host vectors A, B, and C   
    float *h_A = (float *)malloc(size);
    float *h_B = (float *)malloc(size);
    float *h_C = (float *)malloc(size);

    // Initialize host input vectors A and B with random values between 0 and 1.0
    for (int i = 0; i < N; ++i)
    {
        h_A[i] = rand() / (float)RAND_MAX;
        h_B[i] = rand() / (float)RAND_MAX;
    }

    // Call function vectorAdd with timing
    clock_t start = clock();

    vectorAdd(h_A, h_B, h_C, N); // function call
    
    clock_t end = clock();
    double elapsed_ms = (double)(end - start) / CLOCKS_PER_SEC * 1000.0;
    
    // Verify results in ouput vector C is correct
    float errorsum = 0.0f;
    for (int i = 0; i < N; ++i)
    {
        float error = fabs(h_A[i] + h_B[i] - h_C[i]);
        if (error > 1e-5)
        {
            //printf("Result verification failed for element with index %d!\n", i);
            errorsum += error;
        }
    }

    // Print measured function execution time, verification result, and sample elements from each vector
    printf("CPU execution time  : %f ms\n", elapsed_ms);
    printf("Verification result : %s\n", (errorsum > 1e-5) ? "FAILED" : "PASSED");
    printf("Vector samples      : A[0]=%f, B[0]=%f, C[0]=%f\n", h_A[0], h_B[0], h_C[0]);
    
    // Free host memory
    free(h_A);
    free(h_B);
    free(h_C);

    return 0;
}

CPU execution time  : 2.295000 ms
Verification result : PASSED
Vector samples      : A[0]=0.840188, B[0]=0.394383, C[0]=1.234571



---
## 3.2 1D Vector Addition on the Device (GPU)

<img src="images/vectoradd_gpu1.png" width="450" style="float: right; margin-right: 50px;" />

Problem
  - We have three vectors (arrays) `A`, `B`, and `C`, all with `N` elements each.
  - We want to compute the elementwise sum of `A` and `B`, and store the sum in `C`.

We Know
- In CUDA, we have access to many `threads`, where `threads` are organized into `blocks`, and `blocks` are organized into a `grid`.
  - `threadIdx.x` represents a `thread’s index` along the `x` dimension within a `block`. 
  - `blockIdx.x` represents a `block’s index` along the `x` dimension within the `grid`.
  - `blockDim.x` represents the `number of threads` along the `x` dimension with a `block`.
- To get a `thread's global index` on the GPU:
  - `int index = blockDim.x * blockIdx.x + threadIdx.x`
- `Blocks` are assigned to a Streaming Multiprocessor (SM) that has a number of Streaming Processors (SPs).
  - Each `thread` executes its own copy of the `kernel function`, in parallel, with the same parameter values.
  - Each `thread` should process only one element in the arrays using the `index`.
  - If there are more threads than elements (`index >= N`), those threads should `return` immediately from the `kernel function`
- There can be a maximum of `1024` threads in a block.
  - If we have `N = 1048576` elements,
  - and `THREADS_PER_BLOCK = 1024`,
  - we get `BLOCKS = (N + THREADS_PER_BLOCK - 1) / THREADS_PER_BLOCK = (1048576+1024-1) / 1024 = 1024`.
  - And if we have `24` SMs, each will be assigned roughly `1024 / 24 = 42` blocks for maximum efficiency.

<img src="images/vectoradd_gpu2.png" width="450" style="float: right; margin-right: 50px;" />

Solution
1. Define number of elements `N=1048576` and `THREADS_PER_BLOCK=1024`
2. Create a kernel `__global__ void vectorAdd(float *A, float *B, float *C, int n)`
    - Compute global thread ID `idx = blockDim.x * blockIdx.x + threadIdx.x`
    - Return if index is out of bounds (`idx >= n`) which means we have more threads than elements `n`.
      - In this case we won't since `N` is evenly divisible by `THREADS_PER_BLOCK`.
    - Compute `C[idx] = A[idx] + B[idx]`.

3. Create a host function `main(void)`
    - Declare and allocate memory for host vectors `h_A`, `h_B`, and `h_C`.
    - Initialize host vectors `h_A` and `h_B` with `N` random floats each.
    - Declare and allocate memory for device vectors `d_A`, `d_B`, and `d_C`.
    - Copy contents of host vectors `h_A` and `h_B` to device vectors `d_A` and `d_B`.
    - Launch kernel `vectorAdd` with `d_A`, `d_B`, `d_C`, `N`, and measure the execution time for `vectorAdd`.
    - Copy contents of device vector `d_C` to host vector `h_C`.
    - Verify result is correct.
    - Print execution time, verification result, and sample elements in host vectors `h_A`, `h_B`, and `h_C`.
    - Free memory allocated for device vectors `d_A`, `d_B`, and `d_C`.
    - Free memory allocated for host vectors `h_A`, `h_B`, and `h_C`.

<img src="images/coalesced_memory_access.png" width="450" style="float: right; margin-right: 50px;" />

No need for shared or constant memory, and the global memory access pattern is **coalesced** in the code, (a) in the figure.

In [33]:
%%cuda
#include <stdio.h>
#include <cuda_runtime.h>

// Number of elements (1048576)
#define N (1 << 20)

// Number of threads
#define THREADS_PER_BLOCK 1024

// Device kernel (elementwise addition of vectors A and B, placing the sum in vector C)
__global__ void vectorAdd(float *A, float *B, float *C, int n)
{
    // Compute index (idx) from global thread ID
    int idx = blockDim.x * blockIdx.x + threadIdx.x;

    // Return if index is out of bounds (means we have more threads than elements)
    if (idx >= n)
      return;

    // Compute the sum C = A + B for the element with index idx
    C[idx] = A[idx] + B[idx];
}

// Host main routine
int main(void)
{
    // Seed pseudorandom number generator
    srand(0);

    // Compute the size of the vector to use
    size_t size = N * sizeof(float);

    // Allocate the host input vectors A, B, and output vector C
    float *h_A = (float *)malloc(size);
    float *h_B = (float *)malloc(size);
    float *h_C = (float *)malloc(size);

    // Initialize the host input vectors (with random values)
    for (int i = 0; i < N; ++i)
    {
        h_A[i] = rand() / (float)RAND_MAX;
        h_B[i] = rand() / (float)RAND_MAX;
    }

    // Allocate the device input vectors A, B, and output vector  C
    float *d_A, *d_B, *d_C;
    cudaMalloc((void **)&d_A, size);
    cudaMalloc((void **)&d_B, size);
    cudaMalloc((void **)&d_C, size); 

    // Copy the host input vectors A and B in host memory to the device input vectors in device memory
    cudaMemcpy(d_A, h_A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, size, cudaMemcpyHostToDevice);
  
    // Launch kernel with timing
    int blocksPerGrid = (N + THREADS_PER_BLOCK - 1) / THREADS_PER_BLOCK;
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start);
  
    vectorAdd<<<blocksPerGrid, THREADS_PER_BLOCK>>>(d_A, d_B, d_C, N);
  
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    float elapsed;
    cudaEventElapsedTime(&elapsed, start, stop);
 
    // Copy the device result vector in device memory to the host result vector in host memory.
    cudaMemcpy(h_C, d_C, size, cudaMemcpyDeviceToHost);
  
    // Verify the result vector is correct
    float errorsum = 0.0f;
    for (int i = 0; i < N; ++i)
    {
        float error = fabs(h_A[i] + h_B[i] - h_C[i]);
        if (error > 1e-5)
        {
            //fprintf(stderr, "Result verification failed at element %d!\n", i);
            errorsum += error;
        }
    }
  
    // Print measured kernel execution time, verification result, and sample elements from each vector
    printf("GPU execution time  : %f ms\n", elapsed);
    printf("Verification result : %s\n", (errorsum > 1e-5) ? "FAILED" : "PASSED");
    printf("Vector samples      : A[0]=%f, B[0]=%f, C[0]=%f\n", h_A[0], h_B[0], h_C[0]);

    // Free device global memory
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);
  
    // Free host memory
    free(h_A);
    free(h_B);
    free(h_C);

    return 0;
}

GPU execution time  : 0.858848 ms
Verification result : PASSED
Vector samples      : A[0]=0.840188, B[0]=0.394383, C[0]=1.234571



---
## 3.3 1D Convolution on the Host (CPU)

<img src="images/1dconvolution.gif" width="600" style="float: right; margin-right: 50px;" />

Next, let's tackle the problem of a 1-dimensional (1D) convolution.

Problem
- We have an `input` vector, a `kernel` (filter), and an `output` vector.
- We want to slide the `kernel` (filter) over each element in the `input` vector.
- The `kernel` (filter) will be centered over each element in the `input` vector.
- So the `kernel`'s (filter's) width has to be odd, e.g. `1x3`, `1x5`, `1x7`.
- We multiply each element under the `kernel` (filter) in the `input` vector with `kernel`'s (filter's) elements.
- We sum the products, and assign the sum to the `output` vector with the same `index` as the current `input` vector.
- Since the `kernel` (filter) can't be centered over the boundary elements in the `input` vector, we use `zero-padding`.

Solution
1. Define number of elements `N=1048576`
2. Create a function:
   - `void convolve1D(float *input, float *output, float *filter)`
   - Loop through `input` vector.
   - Compute `output[idx] = input[idx + offset] = filter[FILTER_WIDTH/2 + offset]`
     - Only if `if(idx + offset >= 0 && idx + offset < DATA_WIDTH)`
     - Where `offset` ranges from `-FILTER_WIDTH/2` to `+FILTER_WIDTH/2`.
   - This computation is equivalent to
     - Looping through the `input` vector, zero-padded with `FILTER_WIDTH/2` elements on both sides.
     - Centering the `filter` over each original element in the zero-padded `input` vector.
     - Computing the weighted sum and storing it in the `output` vector.
4. Create a function `main(void)`
   - Define a `DATA_WIDTH`, `FILTER_WIDTH` and `FILTER_WIDTH_OFFSET` (which is `FILTER_WIDTH/2`).
   - Declare and allocate memory for vectors `input`, `ouput`, and `filter`.
   - Initialize vector `input` with `DATA_WIDTH` random floats.
   - Initialize vector `filter` with `weights` where each weight is `1.0 / FILTER_WIDTH` (averaging filter).
   - Call function `convolve1D` with `input`, `ouput`, and `filter`.
   - Measure the execution time for `convolve1D`.
   - Print execution time and sample elements in vectors `input` and `output`.
   - Free memory allocated for vectors `input`, `output`, and `filter`.

In [34]:
%%cuda
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <math.h>

// Number of data elements (1048576)
#define DATA_WIDTH (1 << 20)

// Number of filter elements 
#define FILTER_WIDTH 3

// Number of elements on each side of a centered filter
#define FILTER_WIDTH_OFFSET (FILTER_WIDTH / 2)

void convolve1D(float *input, float *output, float *filter)
{
    // Loop through all elements
    for(int d_col = 0; d_col < DATA_WIDTH; d_col++)
    {
        // Apply filter (slide filter over data and compute weighted sum)
        float sum = 0.0f;
        for (int offset_col = -FILTER_WIDTH_OFFSET; offset_col <= FILTER_WIDTH_OFFSET; offset_col++)
        {
            int f_col = FILTER_WIDTH_OFFSET + offset_col; // f_col: 0..FILTER_WIDTH-1
            int i_col = d_col + offset_col;               // i_col: 0-FILTER_WIDTH_OFFSET..DATA_WIDTH-1+FILTER_WIDTH_OFFSET
            
            if(i_col >= 0 && i_col < DATA_WIDTH)
            {
                sum += input[i_col] * filter[f_col];
            }
        }
        
        // Store the weighted sum in the output array
        output[d_col] = sum;
    }
}

int main(void)
{
    // Seed the random number generator
    srand(0);            // use this for same set of random numbers each time the program is run
    //srand(time(NULL)); // use this for different set of random numbers each time the program is run

    // Declare variables
    float *h_input, *h_output, *h_filter; // host copies of input, output, filter
    int data_size = DATA_WIDTH * sizeof(float);     // size of data in bytes
    int filter_size = FILTER_WIDTH * sizeof(float); // size of filter in bytes
   
    // Allocate space for host copies of input, output, filter
    h_input = (float *)malloc(data_size);
    h_output = (float *)malloc(data_size);
    h_filter = (float *)malloc(filter_size);
      
    // Setup input values
    for (int col = 0; col < DATA_WIDTH; col++)
    {
        h_input[col] = (float)rand() / RAND_MAX; // Random floats between 0 and 1.0
    }

    // Setup filter
    for (int col = 0; col < FILTER_WIDTH; col++)
    {
        h_filter[col] = 1.0f / FILTER_WIDTH; // averaging filter
    }
   
    // Call convolve1D() with timing
    clock_t start = clock();                                              // record the start time
    convolve1D(h_input, h_output, h_filter);                              // call convolve1D()
    clock_t stop = clock();                                               // record the stop time
    double elapsed_ms = (double)(stop - start) / CLOCKS_PER_SEC * 1000.0; // calculate the elapsed time in millisecond

    // Print measured calculation execution time
    printf("Calculation (%d elements, 1x%d filter) took %.2f ms\n", DATA_WIDTH, FILTER_WIDTH, elapsed_ms);
   
    // Print out the FILTER_WIDTH number of elements in the two arrays
    printf("Vector samples:\n");
    for(int i = 0; i < FILTER_WIDTH; i++)
    {
        printf("h_input[%d]=%.2f, h_output[%d]=%.2f\n", i, h_input[i], i, h_output[i]);
    }

    // Cleanup
    free(h_input);
    free(h_output);
    free(h_filter);
    
    return 0;
 }

Calculation (1048576 elements, 1x3 filter) took 6.75 ms
Vector samples:
h_input[0]=0.84, h_output[0]=0.41
h_input[1]=0.39, h_output[1]=0.67
h_input[2]=0.78, h_output[2]=0.66



- The output shows:
  - Given an element with index `idx` in the `output` array.
  - It's value is the average of the elements with indices `-FILTER_WIDTH_OFFSET..+FILTER_WIDTH_OFFSET+1` in the `input` array.
    - Since an averaging filter was used.
  - For example
    - If the `FILTER_WIDTH` is `3`, we have `FILTER_WIDTH_OFFSET = FILTER_WIDTH / 2 = 1`.
    - The value of an element with index `idx` in the `output` array is the average of the elements with indices `idx-1`, `idx`, and `idx+1` in the `input` array.
      - `output[idx] = (input[idx-1] +  nput[idx] + nput[idx+1]) / 3`
      - If it's a bounday element, the out-of-bounds indices have zero-padded elements with a value of `0`.

---
## 3.4 1D Convolution on the Device (GPU)

<img src="images/tiled_convolution_1d.png" width="600" style="float: right; margin-right: 50px;" />

Problem
- We have an `input` vector, a `kernel` (filter), and an `output` vector.
- We want to slide the `kernel` (filter) over each element in the `input` vector.
- The `kernel` (filter) will be centered over each element in the `input` vector.
- So the `kernel`'s (filter's) width has to be odd, e.g. `1x3`, `1x5`, `1x7`.
- We multiply each element under the `kernel` (filter) in the `input` vector with `kernel`'s (filter's) elements.
- We sum the products, and assign the sum to the `output` vector with the same `index` as the current `input` vector.
- Since the `kernel` (filter) can't be centered over the boundary elements in the `input` vector, we use `zero-padding`.

Solution
- We have a 1D `input` vector with `N` elements (vector marked with `N` in the figure).
- A `block` of `threads` will process `blockDim` number of elements (top row in figure).
- We don't want to load elements multiple times from `global memory` during the calulation.
  - So each `thread`in a `block` loads its `input` element into `shared memory` (called `Tile` in the figure).
  - The `shared_memory` size needs to be `TILE_BASE_WITH + 2 * FILTER_WIDTH_OFFSET`, where
    - `TILE_BASE_WITH` is the number of original `input` elements in a `block` (highlighted elements in figure).
    - `FILTER_WIDTH_OFFSET` is `FILTER_WIDTH / 2` (called `halo` elements in the figure).
    - `FILTER_WIDTH` is `5` (in the figure).

<img src="images/block_tile_loading_1d.png" width="400" style="float: right; margin-right: 50px;" />

  - This ensures the `filter`, when centered on an element, covers all neighbouring elements, e.g.
    - In `Block 0` the threads use `Tile 0`, where the original elements are `0`, `1`, `2`, `3` (see figure).
    - The `filter` is centered on `0` covering `FILTER_WIDTH_OFFSET` neighbouring elements on each side.
    - For border elements we use zero-padding (called `ghost` elements in the figure for the left-most elements).
    - So the elements included in the first convolution are `ghost`, `ghost`, `0`, `1`, `2` (where `ghost = 0`).
      - When processing element `3`, the `filter` covers elements `1`, `2`, `3`, `4`, `5`.

- For these extra `2 * FILTER_WIDTH_OFFSET` elements to be available in a `block`:
    - The `shared memory`, called `Tile`, needs a size of `TILE_BASE_WITH + 2 * FILTER_WIDTH_OFFSET` (see above).
      - This is the actual size need for a `block` of `threads`, i.e. `blockDim` which includes:
      - Threads for loading the original elements that the `filter` will center on.
      - Threads for the extra `2 * FILTER_WIDTH_OFFSET` border elements.
      - This is illustrated in the bottom figure.
- We also want to load the `filter` elements into `constant` mmeory to avoid hitting global mmeory when accessing them.

- So this is what we'll do:
1. Define:
  - `DATA_WIDTH=1048576` (number of elements in data vectors `input` and `output`)
  - `FILTER_WIDTH=3` (number of elements in the `filter` vector)
  - `FILTER_WIDTH_OFFSET=FILTER_WIDTH/2` (number of elements on each size of a centered `filter`)
  - `TILE_WIDTH_BASE=16` (number original elements in a `block` of `threads`)
    - Where the final tile size is `TILE_WIDTH_BASE + 2 * FILTER_WIDTH_OFFSET` to cover border elements.
    - This is also the size we will use for the `shared memory` and `block` size, i.e. `blockDim.x`.
    - So we have these many `threads` in each `block` and we now a `block` is assigned to an `SM`.
2. Define `constant` memory of size `FILTER_WIDTH` for the `filter`.
3. Create a kernel function `void convolve1D(float *input, float *output)`:
   - Define `shared` memory of size `TILE_WIDTH_BASE + 2 * FILTER_WIDTH_OFFSET`.
   - Let the `threads` in a `block` load their `input` elements into `shared` memory.
     - For border elements, we load the value `0` into `shared` memory (zero-padding).
   - Synchronize `threads`to ensure each `thread`in a `block` has loaded its element into `shared` memory.
   - Compute the convolution as in the CPU solution, but now using `shared` memory (input) and `constant` memory (filter).
   - Store the result in the `ouput` vector.
5. Create a function `main(void)`
   - Declare and allocate memory for vectors `input`, `ouput`, and `filter` on the host (CPU).
   - Declare and allocate memory for vectors `input` and `ouput` on the device (GPU).
   - Initialize vector `input` with `DATA_WIDTH` random floats on the host (CPU).
   - Initialize vector `filter` with `weights` on the host (CPU).
     - Each weight is `1.0 / FILTER_WIDTH` (averaging filter).
   - Copy `input` vector in host (CPU) memory to device (GPU) global memory.
   - Copy `filter` vector in host (CPU) memory to `constant` device (GPU) memory.
   - Launch kernel `convolve1D` with device (GPU) `input` and `ouput` vectors as arguments.
     - Use `gridDim`, `blockDim`, and `shared_memory_size` as launch parameters, where
       - `blockDim = TILE_WIDTH_BASE + 2 * FILTER_WIDTH_OFFSET`
       - `gridDim = (DATA_WIDTH + block_width - 1) / block_width`
       - `shared_mmeory_size = TILE_WIDTH_BASE + 2 * FILTER_WIDTH_OFFSET` (`* sizeof(float)`)
   - Measure the execution time for `convolve1D`.
   - Copy `output` vector in device (GPU) memory to host (CPU) memory.
   - Print execution time and sample elements in vectors `input` and `output`.
   - Free memory allocated for vectors `input`, `output`, and `filter` on the host (CPU).
   - Free memory allocated for vectors `input` and `output` on the device (GPU).

In [35]:
%%cuda
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <cuda_runtime.h>

// Number of data elements (1048576)
#define DATA_WIDTH (1 << 20)

// Number of filter elements
#define FILTER_WIDTH 3

// Number of elements on each side of a centered filter
#define FILTER_WIDTH_OFFSET (FILTER_WIDTH / 2)

// Number of elements in shared memory
#define TILE_WIDTH_BASE 16

// Constant memory with filter weights
__constant__ float filter[FILTER_WIDTH];

// Kernel function
__global__ void convolve1D(float *input, float *output)
{
    // Shared mmeory of size (TILE_WIDTH_BASE + 2 * FILTER_WIDTH_OFFSET) elements
    extern __shared__ float shared[];

    int s_col = threadIdx.x;                           // Thread's index in shared memory
    int d_col = blockIdx.x * blockDim.x + threadIdx.x; // Thread's index in global memory
    int i_col = d_col - FILTER_WIDTH_OFFSET;           // Thread's offset index in global memory

    // Guard against threads with IDs that would index outside the arrays
    if(d_col >= DATA_WIDTH) return;

    // Fill shared memory with elements in global memory
    if (i_col >= 0 && i_col < DATA_WIDTH)
    {
        shared[s_col] = input[i_col];
    }
    else
    {
        shared[s_col] = 0.0f; // zero-padding
    }

    // Make sure each thread in the block has entered its element into shared memory before any thread continues
    __syncthreads();

    // Apply filter
    float sum = 0.0f;
    for (int offset_col = -FILTER_WIDTH_OFFSET; offset_col <= FILTER_WIDTH_OFFSET; offset_col++)
    {
        int f_col = FILTER_WIDTH_OFFSET + offset_col;
        int i_col = s_col + f_col;
        
        if(i_col >= 0 && i_col < blockDim.x)
        {
            sum += shared[i_col] * filter[f_col]; // data elements in shared memory + filter weights in constant memory = super fast computation
        }
    }
    
    // Store the weighted sum in the output array
    output[d_col] = sum;
}

int main(void)
{
    // Seed the random number generator
    srand(0);
    //srand(time(NULL));

    // Declare variables
    float *h_input, *h_output, *h_filter; // host copies of input, output, filter
    float *d_input, *d_output, *d_filter; // device copies of input, output, filter
    int data_size = DATA_WIDTH * sizeof(float);
    int filter_size = FILTER_WIDTH * sizeof(float);
   
    // Allocate space for host (CPU) copies of input, output, filter
    h_input = (float *)malloc(data_size);
    h_output = (float *)malloc(data_size);
    h_filter = (float *)malloc(filter_size);

    // Allocate space for device (GPU) copies of input, output
    cudaMalloc((void **)&d_input, data_size);
    cudaMalloc((void **)&d_output, data_size);
      
    // Setup input values
    for (int col = 0; col < DATA_WIDTH; col++)
    {
        h_input[col] = (float)rand() / RAND_MAX; // Random floats between 0 and 1.0
    }

    // Setup filter
    for (int col = 0; col < FILTER_WIDTH; col++)
    {
        h_filter[col] = 1.0f / FILTER_WIDTH; // averaging filter
    }
   
    // Copy inputs to device
    cudaMemcpy(d_input, h_input, data_size, cudaMemcpyHostToDevice);

    // Copy filter to constant memory
    cudaMemcpyToSymbol(filter, h_filter, filter_size);

    // Calculate launch parameters
    int block_width = TILE_WIDTH_BASE + 2 * FILTER_WIDTH_OFFSET;
    dim3 blockDim(block_width);
    dim3 gridDim((DATA_WIDTH + block_width - 1) / block_width);
    int shared_size = block_width * sizeof(float);

    // Launch kernel convolve1D() with timing
    cudaEvent_t start, stop;         // declare start and stop CUDA events
    cudaEventCreate(&start);         // create start event
    cudaEventCreate(&stop);          // create stop event
    cudaEventRecord(start);          // record the start time

    // Launch convolve1D() kernel on GPU
    convolve1D<<<gridDim, blockDim, shared_size>>>(d_input, d_output); // dynamically sized shared memory
    
    cudaEventRecord(stop);           // record the stop time
    cudaEventSynchronize(stop);      // wait for the stop time to become available
    float elapsed_ms;
    cudaEventElapsedTime(&elapsed_ms, start, stop); // calculate the elapsed time in millisecond

    // Copy result back to host
    cudaMemcpy(h_output, d_output, data_size, cudaMemcpyDeviceToHost);

    // Print measured calculation execution time
    printf("Calculation (%d elements, 1x%d filter) took %.2f ms\n", DATA_WIDTH, FILTER_WIDTH, elapsed_ms);
   
    // Print out the FILTER_WIDTH number of elements in the two arrays
    printf("Vector samples:\n");
    for(int i = 0; i < FILTER_WIDTH; i++)
    {
        printf("h_input[%d]=%.2f, h_output[%d]=%.2f\n", i, h_input[i], i, h_output[i]);
    }
   
    // Cleanup
    free(h_input); free(h_output); free(h_filter); // free host (CPU) memory
    cudaFree(d_input); cudaFree(d_output);         // free device (GPU) memory
    cudaEventDestroy(start);                       // free the CUDA start event
    cudaEventDestroy(stop);                        // free the CUDA stop event
    
    return 0;
}

Calculation (1048576 elements, 1x3 filter) took 0.12 ms
Vector samples:
h_input[0]=0.84, h_output[0]=0.41
h_input[1]=0.39, h_output[1]=0.67
h_input[2]=0.78, h_output[2]=0.66



In the output we see:
- The results are the same for the GPU solution as for the CPU solution.
- The execution time for the GPU solution is significantly fast than the CPU solution.

---
## 3.5 2D Convolution on the Host (CPU)

<img src="images/2dconvolution.gif" width="600" style="float: right; margin-right: 50px;" />

**Note**
  - This is really just the same problem as a 1D convolution, but with an added second dimension.
  - Therefore the problem and solution will be the same, but with the second dimension accounted for.

Problem
- We have an `input` matrix, a `kernel` (filter), and an `output` matrix.
- We want to slide the `kernel` (filter) over each element in the `input` matrix.
- The `kernel` (filter) will be centered over each element in the `input` matrix.
- So the `kernel`'s (filter's) width and height has to be odd, e.g. `3x3`, `5x5`, `7x7`.
- We multiply each element under the `kernel` (filter) in the `input` matrix with the `kernel`'s (filter's) elements.
- We sum the products, and assign the sum to the `output` matrix with the same `index` as the current `input` matrix.
- Since the `kernel` (filter) can't be centered over the boundary elements in the `input` matrix, we use `zero-padding`.

Solution
1. Define:
   - `DATA_WIDTH=32` (number of elements in the `col` dimension for the `input` and `output`)
   - `DATA_HEIGHT=32` (number of elements in the `row` dimension for the `input` and `output`)
   - `FILTER_WIDTH=3` (number of elements int the `col` dimension for the `filter`)
   - `FILTER_HEIGHT=3` (number of elements int the `row` dimension for the `filter`)
   - `FILTER_WIDTH_OFFSET=FILTER_WIDTH/2` (number of elements to the left and right of the centered `filter`)
   - `FILTER_HEIGHT_OFFSET=FILTER_HEIGHT/2` (number of elements above and below the centered `filter`)
3. Create a function:
   - `void convolve2D(float *input, float *output, float *filter)`
   - Loop through `input` matrix.
   - Compute convolution. Store result in `output` matrix.
   - The only difference in the "D convolution compared to the 1D convolution is the additional dimension.
4. Create a function `main(void)`
   - Declare and allocate memory for matrices `input`, `ouput`, and `filter`.
   - Initialize matrix `input` with `DATA_HEIGHT * DATA_WIDTH` random floats.
   - Initialize matrix `filter` with `weights` where each weight is `1.0 / FILTER_HEIGHT * FILTER_WIDTH` (averaging filter).
   - Call function `convolve2D` with `input`, `ouput`, and `filter`.
   - Measure the execution time for `convolve2D`.
   - Print execution time and sample elements in matrices `input` and `output`.
   - Free memory allocated for matrices `input`, `output`, and `filter`.

In [36]:
%%cuda
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <math.h>

#define DATA_WIDTH 32
#define DATA_HEIGHT 32
#define FILTER_WIDTH 3
#define FILTER_HEIGHT 3
#define FILTER_WIDTH_OFFSET (FILTER_WIDTH/2)
#define FILTER_HEIGHT_OFFSET (FILTER_HEIGHT/2)

void convolve2D(float *input, float *output, float *filter)
{
    for(int d_row = 0; d_row < DATA_HEIGHT; d_row++)
    {
        for(int d_col = 0; d_col < DATA_WIDTH; d_col++)
        {
            float sum = 0.0f;
            for (int offset_row = -FILTER_HEIGHT_OFFSET; offset_row <= FILTER_HEIGHT_OFFSET; offset_row++)
            {
                for (int offset_col = -FILTER_WIDTH_OFFSET; offset_col <= FILTER_WIDTH_OFFSET; offset_col++)
                {
                    int f_row = FILTER_HEIGHT_OFFSET + offset_row;
                    int f_col = FILTER_WIDTH_OFFSET + offset_col;
                    int i_row = d_row + offset_row;
                    int i_col = d_col + offset_col;

                    if(i_row >= 0 && i_row < DATA_HEIGHT && i_col >= 0 && i_col < DATA_WIDTH)
                    {
                        sum += input[i_row * DATA_WIDTH + i_col] * filter[f_row * FILTER_WIDTH + f_col];
                    }
                }
            }

            output[d_row * DATA_WIDTH + d_col] = sum;
        }
    }
}

int main(void)
{
    srand(0);

    float *h_input = (float *)malloc(DATA_WIDTH * DATA_HEIGHT * sizeof(float));
    float *h_output = (float *)malloc(DATA_WIDTH * DATA_HEIGHT * sizeof(float));
    float *h_filter = (float *)malloc(FILTER_WIDTH * FILTER_HEIGHT * sizeof(float));

    for(int row = 0; row < DATA_HEIGHT; row++)
    {
        for(int col = 0; col < DATA_WIDTH; col++)
        {
            h_input[row * DATA_WIDTH + col] = (float)rand() / RAND_MAX;
        }
    }

    for(int row = 0; row < FILTER_HEIGHT; row++)
    {
        for(int col = 0; col < FILTER_WIDTH; col++)
        {
            h_filter[row * FILTER_WIDTH + col] = 1.0f / (FILTER_WIDTH * FILTER_HEIGHT);
        }
    }

    // Call convolve2D() with timing
    clock_t start = clock();
    convolve2D(h_input, h_output, h_filter);
    clock_t stop = clock();
    double elapsed_ms = (double)(stop - start) / CLOCKS_PER_SEC * 1000.0;

    printf("Calculation (%d elements, %dx%d filter) took %.2f ms\n", DATA_HEIGHT * DATA_WIDTH, FILTER_HEIGHT, FILTER_WIDTH, elapsed_ms);
    printf("\nMatrix samples:\n");
    printf("h_input %-12s h_output\n", "");
    for(int row = 0; row < FILTER_HEIGHT; row++)
    {
        for(int col = 0; col < FILTER_WIDTH; col++)
        {
            printf("%.3f ", h_input[row * DATA_WIDTH + col]);
        }
        printf("%-3s","");
        for(int col = 0; col < FILTER_WIDTH; col++)
        {
            printf("%.3f ", h_output[row * DATA_WIDTH + col]);
        }
        printf("\n");
    }

    free(h_input);
    free(h_output);
    free(h_filter);

    return 0;
}

Calculation (1024 elements, 3x3 filter) took 0.06 ms

Matrix samples:
h_input              h_output
0.840 0.394 0.783    0.238 0.396 0.382 
0.613 0.296 0.638    0.328 0.527 0.568 
0.267 0.540 0.375    0.325 0.514 0.616 



- The output shows:
  - Given an element with index `[row, col]` in the `output` matrix.
  - It's value is the average of the elements with indices:
    - `-FILTER_HEIGHT_OFFSET..+FILTER_HEIGHT_OFFSET+1` in the `input` matrix's `row`.
    - `-FILTER_WIDTH_OFFSET..+FILTER_WIDTH_OFFSET+1` in the `input` matrix's `col`.
    - Since an averaging filter was used.
  - For example
    - If the `FILTER_HEIGHT` is `3`, we have `FILTER_HEIGHT_OFFSET = FILTER_HEIGHT / 2 = 1`.
    - If the `FILTER_WIDTH` is `3`, we have `FILTER_WIDTH_OFFSET = FILTER_WIDTH / 2 = 1`.
    - The value of an element with index `[row, col]` in the `output` matrix is the average of the elements in the `input` matrix with indices:

      ```c
      [row-1, col-1]  [row-1, col]  [row-1, col+1]
      [row  , col-1]  [row  , col]  [row  , col+1]
      [row+1, col-1]  [row+1, col]  [row+1, col+1]
      ```
    - If it's a bounday element, the out-of-bounds indices have zero-padded elements with a value of `0`.

---
## 3.6 2D Convolution on the Device (GPU)

**Note**
  - This is really just the same problem as a 1D convolution, but with an added second dimension.
  - Therefore the problem and solution will be the same, but with the second dimension accounted for.

Problem
- We have an `input` matrix, a `kernel` (filter), and an `output` matrix.
- We want to slide the `kernel` (filter) over each element in the `input` matrix.
- The `kernel` (filter) will be centered over each element in the `input` matrix.
- So the `kernel`'s (filter's) height and width has to be odd, e.g. `3x3`, `5x5`, `7x7`.
- We multiply each element under the `kernel` (filter) in the `input` matrix with the `kernel`'s (filter's) elements.
- We sum the products, and assign the sum to the `output` matrix with the same `index` as the current `input` matrix.
- Since the `kernel` (filter) can't be centered over the boundary elements in the `input` matrix, we use `zero-padding`.

Solution
1. Define:
  -  `DATA_WIDTH=32` (number of elements in the `col` dimension for the `input` and `output`)
   - `DATA_HEIGHT=32` (number of elements in the `row` dimension for the `input` and `output`)
   - `FILTER_WIDTH=3` (number of elements int the `col` dimension for the `filter`)
   - `FILTER_HEIGHT=3` (number of elements int the `row` dimension for the `filter`)
   - `FILTER_WIDTH_OFFSET=FILTER_WIDTH/2` (number of elements to the left and right of the centered `filter`)
   - `FILTER_HEIGHT_OFFSET=FILTER_HEIGHT/2` (number of elements above and below the centered `filter`) 
   - `TILE_WIDTH_BASE=16` (number original elements in a `block` of `threads` in the `col` dimension)
   - `TILE_HEIGHT_BASE=16` (number original elements in a `block` of `threads` in the `row` dimension)
     - Where the final tile size in the `col` dimension is `TILE_WIDTH_BASE + 2 * FILTER_WIDTH_OFFSET` to cover left and right border elements.
       - This is also the size we will use for the `col` dimension in `shared memory` and `block` size, i.e. `blockDim.x`.
     - Where the final tile size in the `row` dimension is `TILE_HEIGHT_BASE + 2 * FILTER_HEIGHT_OFFSET` to cover top and bottom border elements.
       - This is also the size we will use for the `row` dimension in `shared memory` and `block` size, i.e. `blockDim.y`.
     - So we have these many 2D `threads` in each `block` and we know a `block` is assigned to an `SM`.
3. Define `constant` memory of size `FILTER_HEIGHT * FILTER_WIDTH` for the `filter`.
4. Create a kernel function `void convolve2D(float *input, float *output)`:
   - Define `shared` memory of size `(TILE_HEIGHT_BASE + 2 * FILTER_HEIGHT_OFFSET) * (TILE_WIDTH_BASE + 2 * FILTER_WIDTH_OFFSET)`.
   - Let the `threads` in a `block` load their `input` elements into `shared` memory.
     - For border elements, we load the value `0` into `shared` memory (zero-padding).
   - Synchronize `threads`to ensure each `thread`in a `block` has loaded its element into `shared` memory.
   - Compute the convolution as in the CPU solution, but now using `shared` memory (input) and `constant` memory (filter).
   - Store the result in the `ouput` matrix.
5. Create a function `main(void)`
   - Declare and allocate memory for matrices `input`, `ouput`, and `filter` on the host (CPU).
   - Declare and allocate memory for matrices `input` and `ouput` on the device (GPU).
   - Initialize matrix `input` with `DATA_HEIGHT * DATA_WIDTH` random floats on the host (CPU).
   - Initialize matrix `filter` with `weights` on the host (CPU).
     - Each weight is `1.0 / (FILTER_HEIGHT * FILTER_WIDTH)` (averaging filter).
   - Copy `input` matrix in host (CPU) memory to device (GPU) global memory.
   - Copy `filter` matrix in host (CPU) memory to `constant` device (GPU) memory.
   - Launch kernel `convolve2D` with device (GPU) `input` and `ouput` matrices as arguments.
     - Use `gridDim`, `blockDim`, and `shared_memory_size` as launch parameters, where
       - `blockDim.x = TILE_WIDTH_BASE + 2 * FILTER_WIDTH_OFFSET`
       - `blockDim.y = TILE_HEIGHT_BASE + 2 * FILTER_HEIGHT_OFFSET`
       - `gridDim.x = (DATA_WIDTH + block_width - 1) / block_width`
       - `gridDim.y = (DATA_HEIGHT + block_height - 1) / block_height`
       - `shared_mmeory_size = (TILE_WIDTH_BASE + 2 * FILTER_WIDTH_OFFSET) * (TILE_HEIGHT_BASE + 2 * FILTER_HEIGHT_OFFSET)` (`* sizeof(float)`)
   - Measure the execution time for `convolve2D`.
   - Copy `output` matrix in device (GPU) memory to host (CPU) memory.
   - Print execution time and sample elements in matrices `input` and `output`.
   - Free memory allocated for matrices `input`, `output`, and `filter` on the host (CPU).
   - Free memory allocated for matrices `input` and `output` on the device (GPU).

In [37]:
%%cuda
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <cuda_runtime.h>

#define DATA_WIDTH 32
#define DATA_HEIGHT 32
#define FILTER_WIDTH 3
#define FILTER_HEIGHT 3
#define FILTER_WIDTH_OFFSET (FILTER_WIDTH/2)
#define FILTER_HEIGHT_OFFSET (FILTER_HEIGHT/2)
#define TILE_WIDTH_BASE 16
#define TILE_HEIGHT_BASE 16

__constant__ float filter[FILTER_WIDTH * FILTER_HEIGHT];

__global__ void convolve2D(float *input, float *output)
{
    extern __shared__ float shared[];

    int s_row = threadIdx.y;
    int s_col = threadIdx.x;
    
    int d_col = blockIdx.x * blockDim.x + threadIdx.x;
    int d_row = blockIdx.y * blockDim.y + threadIdx.y;

    int i_row = d_row - FILTER_HEIGHT_OFFSET;
    int i_col = d_col - FILTER_WIDTH_OFFSET;

    if(d_col >= DATA_WIDTH || d_row >= DATA_HEIGHT) return;   
    
    if (i_row >= 0 && i_row < DATA_HEIGHT && i_col >= 0 && i_col < DATA_WIDTH)
    {
        shared[s_row * blockDim.x + s_col] = input[i_row * DATA_WIDTH + i_col];
    }
    else
    {
        shared[s_row * blockDim.x + s_col] = 0.0f; // zero-padding
    }

    __syncthreads();

    float sum = 0.0f;
    for (int offset_row = -FILTER_HEIGHT_OFFSET; offset_row <= FILTER_HEIGHT_OFFSET; offset_row++)
    {
        for (int offset_col = -FILTER_WIDTH_OFFSET; offset_col <= FILTER_WIDTH_OFFSET; offset_col++)
        {
            int f_row = FILTER_HEIGHT_OFFSET + offset_row;
            int f_col = FILTER_WIDTH_OFFSET + offset_col;
            int i_row = s_row + f_row;
            int i_col = s_col + f_col;

            if(i_row >= 0 && i_row < blockDim.y && i_col >= 0 && i_col < blockDim.x)
            {
                sum += shared[i_row * blockDim.x + i_col] * filter[f_row * FILTER_WIDTH + f_col];
            }
        }
    }
    output[d_row * DATA_WIDTH + d_col] = sum;
}

int main(void)
{
    srand(0);

    float *h_input, *h_output, *h_filter;
    float *d_input, *d_output, *d_filter;
    int data_size = DATA_WIDTH * DATA_HEIGHT * sizeof(float);
    int filter_size = FILTER_WIDTH * FILTER_HEIGHT * sizeof(float);

    h_input = (float *)malloc(data_size);
    h_output = (float *)malloc(data_size);
    h_filter = (float *)malloc(filter_size);
    
    cudaMalloc((void **)&d_input, data_size);
    cudaMalloc((void **)&d_output, data_size);
    cudaMalloc((void **)&d_filter, filter_size);

    for(int row = 0; row < DATA_HEIGHT; row++)
    {
        for(int col = 0; col < DATA_WIDTH; col++)
        {
            h_input[row * DATA_WIDTH + col] = (float)rand() / RAND_MAX;
        }
    }

    for(int row = 0; row < FILTER_HEIGHT; row++)
    {
        for(int col = 0; col < FILTER_WIDTH; col++)
        {
            h_filter[row * FILTER_WIDTH + col] = 1.0f / (FILTER_WIDTH * FILTER_HEIGHT);
        }
    }

    cudaMemcpy(d_input, h_input, data_size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_output, h_output, data_size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_filter, h_filter, filter_size, cudaMemcpyHostToDevice);
    cudaMemcpyToSymbol(filter, h_filter, filter_size);
    
    int block_size_width = TILE_WIDTH_BASE + 2 * FILTER_WIDTH_OFFSET;
    int block_size_height = TILE_HEIGHT_BASE + 2 * FILTER_HEIGHT_OFFSET;

    dim3 blockDim(block_size_width, block_size_height);
    dim3 gridDim((DATA_WIDTH + block_size_width - 1) / block_size_width, (DATA_HEIGHT + block_size_height - 1) / block_size_height);
    int shared_size = block_size_width * block_size_height * sizeof(float);

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start);

    // Launch convolve2D() kernel on GPU
    convolve2D<<<gridDim, blockDim, shared_size>>>(d_input, d_output);
    
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    float elapsed_ms;
    cudaEventElapsedTime(&elapsed_ms, start, stop);

    cudaMemcpy(h_output, d_output, data_size, cudaMemcpyDeviceToHost);

    printf("Calculation (%d elements, %dx%d filter) took %.2f ms\n", DATA_HEIGHT * DATA_WIDTH, FILTER_HEIGHT, FILTER_WIDTH, elapsed_ms);
    printf("\nMatrix samples:\n");
    printf("h_input %-12s h_output\n", "");
    for(int row = 0; row < FILTER_HEIGHT; row++)
    {
        for(int col = 0; col < FILTER_WIDTH; col++)
        {
            printf("%.3f ", h_input[row * DATA_WIDTH + col]);
        }
        printf("%-3s","");
        for(int col = 0; col < FILTER_WIDTH; col++)
        {
            printf("%.3f ", h_output[row * DATA_WIDTH + col]);
        }
        printf("\n");
    }

    cudaFree(d_input);
    cudaFree(d_output);
    cudaFree(d_filter);

    free(h_input);
    free(h_output);
    free(h_filter);

    return 0;
}

Calculation (1024 elements, 3x3 filter) took 0.02 ms

Matrix samples:
h_input              h_output
0.840 0.394 0.783    0.238 0.396 0.382 
0.613 0.296 0.638    0.328 0.527 0.568 
0.267 0.540 0.375    0.325 0.514 0.616 



In the output we see:
- The results are the same for the GPU solution as for the CPU solution.
- The execution time for the GPU solution is significantly fast than the CPU solution.

---
# 4. Cleanup
---

- Let's remove all files that have been created by this notebook.

In [38]:
import os, shutil

dirs = ["nvcc4jupyter", "src", "include", "bin", ".vscode"]
files = ["main.cu", "main.exe", "profile_report.ncu-rep"]

for d in dirs:
    if os.path.exists(d):
        shutil.rmtree(d)

for f in files:
    if os.path.exists(f):
        os.remove(f)